# やまぴーさん part

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import torch
import importlib
import cv2 
import pandas as pd

from PIL import Image
from IPython.display import display

In [ ]:
%%time

%cp -r /kaggle/input/yolox-pet2 /kaggle/working/
%cd /kaggle/working/yolox-pet2/yolox-dep

In [ ]:
%%time

!pip install pip-21.3.1-py3-none-any.whl -f ./ --no-index
!pip install loguru-0.5.3-py3-none-any.whl -f ./ --no-index
!pip install ninja-1.10.2.3-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl -f ./ --no-index
!pip install onnx-1.8.1-cp37-cp37m-manylinux2010_x86_64.whl -f ./ --no-index
!pip install onnxruntime-1.8.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl -f ./ --no-index
!pip install onnxoptimizer-0.2.6-cp37-cp37m-manylinux2014_x86_64.whl -f ./ --no-index
!pip install thop-0.0.31.post2005241907-py3-none-any.whl -f ./ --no-index
!pip install tabulate-0.8.9-py3-none-any.whl -f ./ --no-index

In [ ]:
%%time

%cd /kaggle/working/yolox-pet2/YOLOX
!pip install -r requirements.txt
!pip install -v -e . 

In [ ]:
%%time

# Install CocoAPI tool
%cd /kaggle/working/yolox-pet2/yolox-dep/cocoapi/PythonAPI

!make
!make install
!python setup.py install

In [ ]:
import pycocotools

In [ ]:
import cv2
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import torchvision.transforms as T
from torchvision.io import ImageReadMode, read_image, write_jpeg
import gc

In [ ]:
%cd /kaggle/working/

In [ ]:
df = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
# df = pd.read_csv("../input/petfinder-pawpularity-score/train.csv").sample(6800)

In [ ]:
# df = pd.concat([df]*900, axis="index").reset_index(drop=True)
# df.shape

In [ ]:
# df.head()

In [ ]:
# from tqdm import tqdm_notebook as tqdm
# import shutil

# !mkdir /kaggle/working/copy_train
# for file in tqdm(df["Id"]):
#     shutil.copyfile(f"../input/petfinder-pawpularity-score/train/{file}.jpg", f"/kaggle/working/copy_train/{file}.jpg")

In [ ]:
df["Id"] = "../input/petfinder-pawpularity-score/test/" + df["Id"] + ".jpg"
# df["Id"] = "../input/petfinder-pawpularity-score/train/" + df["Id"] + ".jpg"
# df["Id"].head()

In [ ]:
!mkdir /kaggle/working/test_center_crop/

In [ ]:
def center_crop(Id: str):
    image = read_image(Id, mode=ImageReadMode.RGB)
    write_jpeg(
        T.CenterCrop(min(image.shape[1:]))(image),
        f'/kaggle/working/test_center_crop/{Id.replace("../input/petfinder-pawpularity-score/test/", "")}',
#         f'/kaggle/working/test_center_crop/{Id.replace("../input/petfinder-pawpularity-score/train/", "")}',
        quality=100
    )

In [ ]:
%%time
df["Id"].parallel_apply(center_crop)

In [ ]:
del df
gc.collect()

In [ ]:
%cd /kaggle/working/yolox-pet2/YOLOX

In [ ]:
import argparse
import os
import time

import numpy as np
from loguru import logger
# import jpeg4py as jpeg

import cv2

import torch

from yolox.data.data_augment import ValTransform
from yolox.data.datasets import COCO_CLASSES
from yolox.exp import get_exp
from yolox.utils import fuse_model, get_model_info, postprocess, vis

IMAGE_EXT = [".jpg", ".jpeg", ".webp", ".bmp", ".png"]

In [ ]:
def make_parser():
    parser = argparse.ArgumentParser("YOLOX Demo!")
    parser.add_argument(
        "demo", default="image", help="demo type, eg. image, video and webcam"
    )
    parser.add_argument("-expn", "--experiment-name", type=str, default=None)
    parser.add_argument("-n", "--name", type=str, default=None, help="model name")

    parser.add_argument(
        "--path", default="./assets/dog.jpg", help="path to images or video"
    )
    parser.add_argument("--camid", type=int, default=0, help="webcam demo camera id")
    parser.add_argument(
        "--save_result",
        action="store_true",
        help="whether to save the inference result of image/video",
    )

    # exp file
    parser.add_argument(
        "-f",
        "--exp_file",
        default=None,
        type=str,
        help="pls input your experiment description file",
    )
    parser.add_argument("-c", "--ckpt", default=None, type=str, help="ckpt for eval")
    parser.add_argument(
        "--device",
        default="cpu",
        type=str,
        help="device to run our model, can either be cpu or gpu",
    )
    parser.add_argument("--conf", default=0.3, type=float, help="test conf")
    parser.add_argument("--nms", default=0.3, type=float, help="test nms threshold")
    parser.add_argument("--tsize", default=None, type=int, help="test img size")
    parser.add_argument(
        "--fp16",
        dest="fp16",
        default=False,
        action="store_true",
        help="Adopting mix precision evaluating.",
    )
    parser.add_argument(
        "--legacy",
        dest="legacy",
        default=False,
        action="store_true",
        help="To be compatible with older versions",
    )
    parser.add_argument(
        "--fuse",
        dest="fuse",
        default=False,
        action="store_true",
        help="Fuse conv and bn for testing.",
    )
    parser.add_argument(
        "--trt",
        dest="trt",
        default=False,
        action="store_true",
        help="Using TensorRT model for testing.",
    )
    return parser


def get_image_list(path):
    image_names = []
    for maindir, subdir, file_name_list in os.walk(path):
        for filename in file_name_list:
            apath = os.path.join(maindir, filename)
            ext = os.path.splitext(apath)[1]
            if ext in IMAGE_EXT:
                image_names.append(apath)
    return image_names


class Predictor(object):
    def __init__(
        self,
        model,
        exp,
        cls_names=COCO_CLASSES,
        trt_file=None,
        decoder=None,
        device="cpu",
        fp16=False,
        legacy=False,
    ):
        self.model = model
        self.cls_names = cls_names
        self.decoder = decoder
        self.num_classes = exp.num_classes
        self.confthre = exp.test_conf
        self.nmsthre = exp.nmsthre
        self.test_size = exp.test_size
        self.device = device
        self.fp16 = fp16
        self.preproc = ValTransform(legacy=legacy)
        if trt_file is not None:
            from torch2trt import TRTModule

            model_trt = TRTModule()
            model_trt.load_state_dict(torch.load(trt_file))

            x = torch.ones(1, 3, exp.test_size[0], exp.test_size[1]).cuda()
            self.model(x)
            self.model = model_trt

    def inference(self, img):
        img_info = {"id": 0}
        if isinstance(img, str):
            img_info["file_name"] = os.path.basename(img)
            img = cv2.imread(img)
#             img = jpeg.JPEG(img).decode()[:, :, ::-1]
        else:
            img_info["file_name"] = None

        height, width = img.shape[:2]
        img_info["height"] = height
        img_info["width"] = width
        img_info["raw_img"] = img

        ratio = min(self.test_size[0] / img.shape[0], self.test_size[1] / img.shape[1])
        img_info["ratio"] = ratio

        img, _ = self.preproc(img, None, self.test_size)
        img = torch.from_numpy(img).unsqueeze(0)
        img = img.float()
        if self.device == "gpu":
            img = img.cuda()
            if self.fp16:
                img = img.half()  # to FP16

        with torch.no_grad():
            t0 = time.time()
            outputs = self.model(img)
            if self.decoder is not None:
                outputs = self.decoder(outputs, dtype=outputs.type())
            outputs = postprocess(
                outputs, self.num_classes, self.confthre,
                self.nmsthre, class_agnostic=True
            )
            logger.info("Infer time: {:.4f}s".format(time.time() - t0))
        return outputs, img_info

    def visual(self, output, img_info, cls_conf=0.35):
        ratio = img_info["ratio"]
        img = img_info["raw_img"]
        if output is None:
            return img
        output = output.cpu()

        bboxes = output[:, 0:4]

        # preprocessing: resize
        bboxes /= ratio

        cls = output[:, 6]
        scores = output[:, 4] * output[:, 5]

        vis_res = vis(img, bboxes, scores, cls, cls_conf, self.cls_names)
        return vis_res

    def crop_max_dog_or_cat(self, output, img_info):
        ratio = img_info["ratio"]
        img = img_info["raw_img"]
        if output is None:
            return img, None
        output = output.cpu()

        bboxes = output[:, 0:4]

        # preprocessing: resize
        bboxes /= ratio
        bboxes = bboxes.numpy()

        cls = output[:, 6]
        cls = cls.numpy().astype(int)
        cls_names = [self.cls_names[clsid] for clsid in cls]
        scores = (output[:, 4] * output[:, 5]).numpy()

        img_bbox_infos = dict()
        img_bbox_infos["file_name"] = img_info["file_name"]
        img_bbox_infos["height"] = img_info["height"]
        img_bbox_infos["width"] = img_info["width"]
        img_bbox_infos["cls_names"] = cls_names
        img_bbox_infos["scores"] = scores
        img_bbox_infos["cls_ids"] = cls
        img_bbox_infos["bboxes"] = bboxes

        dog_or_cat_idx = [i for i, x in enumerate(cls_names) if x in ["dog", "cat"]]
        if len(dog_or_cat_idx) < 1:
            return img, img_bbox_infos

        dog_or_cat_scores = scores[dog_or_cat_idx]
        dog_or_cat_bboxes = bboxes[dog_or_cat_idx]

        max_score_idx = np.argmax(dog_or_cat_scores)
        max_score_bbox = dog_or_cat_bboxes[max_score_idx]
        x0 = max(0, int(max_score_bbox[0]))
        y0 = max(0, int(max_score_bbox[1]))
        x1 = min(int(max_score_bbox[2]), img_info["width"])
        y1 = min(int(max_score_bbox[3]), img_info["height"])

        res_img = img[y0:y1, x0:x1]

        return res_img, img_bbox_infos

In [ ]:
import pickle
import sys

def save_pickle(obj, file_path):
    max_bytes = 2 ** 31 - 1
    bytes_out = pickle.dumps(obj, protocol=pickle.HIGHEST_PROTOCOL)
    n_bytes = sys.getsizeof(bytes_out)
    with open(file_path, "wb") as f_out:
        for idx in range(0, n_bytes, max_bytes):
            f_out.write(bytes_out[idx : idx + max_bytes])

def load_pickle(file_path):
    max_bytes = 2 ** 31 - 1
    input_size = os.path.getsize(file_path)
    bytes_in = bytearray(0)
    with open(file_path, "rb") as f_in:
        for _ in range(0, input_size, max_bytes):
            bytes_in += f_in.read(max_bytes)
    obj = pickle.loads(bytes_in)
    return obj

In [ ]:
def image_crop(predictor, vis_folder, path, current_time, save_result):
    if os.path.isdir(path):
        files = get_image_list(path)
    else:
        files = [path]
    files.sort()
    for image_name in files:
        outputs, img_info = predictor.inference(image_name)
        result_image, bboxes_info = predictor.crop_max_dog_or_cat(outputs[0], img_info)
        if save_result:
            save_folder = os.path.join(
                vis_folder, "test_images"
            )
            os.makedirs(save_folder, exist_ok=True)
            save_file_name = os.path.join(save_folder, os.path.basename(image_name))
            logger.info("Saving detection result in {}".format(save_file_name))
            cv2.imwrite(save_file_name, result_image)
            if bboxes_info is not None:
                save_pickle(bboxes_info, f"{str(save_file_name)}.pkl")


In [ ]:
!pwd

In [ ]:
# args = make_parser().parse_args(args=["image_clop", "-n", "yolox-l", "-c", "weights/yolox_l.pth", "--path", "/kaggle/working/test_center_crop/", "--conf", "0.25", "--nms", "0.45", "--tsize", "640", "--save_result", "--device", "gpu"])
args = make_parser().parse_args(args=["image_clop", "-n", "yolox-x", "-c", "weights/yolox_x.pth", "--path", "/kaggle/input/petfinder-pawpularity-score/test/", "--conf", "0.01", "--nms", "0.4", "--tsize", "640", "--save_result", "--device", "gpu"])
# args = make_parser().parse_args(args=["image_clop", "-n", "yolox-x", "-c", "weights/yolox_x.pth", "--path", "/kaggle/working/copy_train/", "--conf", "0.01", "--nms", "0.4", "--tsize", "640", "--save_result", "--device", "gpu"])
args

In [ ]:
exp = get_exp(args.exp_file, args.name)
exp.seed = 1031
exp.output_dir = "/kaggle/working/"
exp.exp_name = "yolox_x/test_images/"
exp

In [ ]:
if not args.experiment_name:
    args.experiment_name = exp.exp_name

file_name = os.path.join(exp.output_dir, args.experiment_name)
os.makedirs(file_name, exist_ok=True)

vis_folder = None
if args.save_result:
    vis_folder = os.path.join(file_name, "vis_res")
    os.makedirs(vis_folder, exist_ok=True)

if args.trt:
    args.device = "gpu"

logger.info("Args: {}".format(args))

if args.conf is not None:
    exp.test_conf = args.conf
if args.nms is not None:
    exp.nmsthre = args.nms
if args.tsize is not None:
    exp.test_size = (args.tsize, args.tsize)

model = exp.get_model()
logger.info("Model Summary: {}".format(get_model_info(model, exp.test_size)))

if args.device == "gpu":
    model.cuda()
    if args.fp16:
        model.half()  # to FP16
model.eval()

if not args.trt:
    if args.ckpt is None:
        ckpt_file = os.path.join(file_name, "best_ckpt.pth")
    else:
        ckpt_file = args.ckpt
    logger.info("loading checkpoint")
    ckpt = torch.load(ckpt_file, map_location="cpu")
    # load the model state dict
    model.load_state_dict(ckpt["model"])
    logger.info("loaded checkpoint done.")

if args.fuse:
    logger.info("\tFusing model...")
    model = fuse_model(model)

if args.trt:
    assert not args.fuse, "TensorRT model is not support model fusing!"
    trt_file = os.path.join(file_name, "model_trt.pth")
    assert os.path.exists(
        trt_file
    ), "TensorRT model is not found!\n Run python3 tools/trt.py first!"
    model.head.decode_in_inference = False
    decoder = model.head.decode_outputs
    logger.info("Using TensorRT to inference")
else:
    trt_file = None
    decoder = None

predictor = Predictor(
    model, exp, COCO_CLASSES, trt_file, decoder,
    args.device, args.fp16, args.legacy,
)
current_time = time.localtime()
image_crop(predictor, vis_folder, args.path, current_time, args.save_result)

In [ ]:
from glob import glob

bbox_df = pd.DataFrame().from_dict(
    [
        load_pickle(filepath) for filepath in glob(f"/kaggle/working/yolox_x/test_images/vis_res/test_images/*.jpg.pkl")
    ]
)
bbox_df["Id"] = bbox_df["file_name"].str.replace(".jpg", "")
bbox_df["aspect_ratio"] = bbox_df["height"] / bbox_df["width"]
bbox_df["area"] = bbox_df["height"] * bbox_df["width"]
bbox_df["num_dog"] = bbox_df["cls_names"].astype(str).str.count("dog")
bbox_df["num_cat"] = bbox_df["cls_names"].astype(str).str.count("cat")
bbox_df["num_teddy_bear"] = bbox_df["cls_names"].astype(str).str.count("teddy bear")
bbox_df["num_person"] = bbox_df["cls_names"].astype(str).str.count("person")
bbox_df["num_dog_cat"] = bbox_df["num_dog"] + bbox_df["num_cat"]
bbox_df["num_dog_cat_teddy_bear"] = bbox_df["num_dog"] + bbox_df["num_cat"] + bbox_df["num_teddy_bear"]
bbox_df

In [ ]:
import gc

bbox_df.to_csv("/kaggle/working/bbox_info.csv", index=False)
del bbox_df
gc.collect()

In [ ]:
!rm -rf /kaggle/working/yolox_x/test_images/vis_res/test_images/*.jpg.pkl

In [ ]:
%cd /kaggle/working/

In [ ]:
%%time

import gc
import os
import sys
import warnings
from pathlib import Path
from pprint import pprint

import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import torchvision.transforms as T
from torchvision.io import ImageReadMode, read_image
from pytorch_lightning import callbacks, seed_everything, LightningDataModule
from torch.utils.data import DataLoader, Dataset

!pip install ../input/omegaconf/omegaconf-2.0.5-py3-none-any.whl
from omegaconf import OmegaConf
sys.path.append('../input/timm-3monthsold/pytorch-image-models-master 2')
import timm

warnings.filterwarnings("ignore")

In [ ]:
%%time

!pip install ../input/python-box/python_box-5.4.1-py3-none-any.whl
from box import Box

In [ ]:
GBDT_EXP_NUM = "065"

In [ ]:
gbdt_config = OmegaConf.load(
    f"../input/petfinder-pawpularity-score-xgb-exp-{GBDT_EXP_NUM}/output.json"
)
# pprint(gbdt_config)

In [ ]:
gbdt_config["features"]

In [ ]:
PAW_EXP_NUM = gbdt_config["features"]["paw_embed"].replace("exp_", "")
BIN_PAW_EXP_NUM = gbdt_config["features"]["bin_paw"].replace("exp_", "")
AGE_EXP_NUM = gbdt_config["features"]["age"].replace("exp_", "")
BREED_EXP_NUM = gbdt_config["features"]["breed"].replace("exp_", "")
ADOPTION_SPEED_EXP_NUM = gbdt_config["features"]["adoption_speed"].replace("exp_", "")
GENDER_EXP_NUM = gbdt_config["features"]["gender"].replace("exp_", "")
MATURITY_SIZE_EXP_NUM = gbdt_config["features"]["maturity_size"].replace("exp_", "")
BIN_SWIN_EXP_NUM = "109"
CENTER_CROP_SWIN_EXP_NUM = "101"

In [ ]:
paw_config = OmegaConf.load(
    f"../input/petfinder-pawpularity-score-exp-{PAW_EXP_NUM}/exp_{PAW_EXP_NUM}.yaml"
)
paw_config.val_loader.batch_size = 128
pprint(paw_config)

In [ ]:
bin_paw_config = OmegaConf.load(
    f"../input/petfinder-pawpularity-score-exp-{BIN_PAW_EXP_NUM}/exp_{BIN_PAW_EXP_NUM}.yaml"
)
bin_paw_config.val_loader.batch_size = 128
pprint(bin_paw_config)

In [ ]:
age_config = OmegaConf.load(
    f"../input/petfinder-pawpularity-score-exp-{AGE_EXP_NUM}/exp_{AGE_EXP_NUM}.yaml"
)
age_config.val_loader.batch_size = 128
pprint(age_config)

In [ ]:
breed_config = OmegaConf.load(
    f"../input/petfinder-pawpularity-score-exp-{BREED_EXP_NUM}/exp_{BREED_EXP_NUM}.yaml"
)
breed_config.val_loader.batch_size = 128
pprint(breed_config)

In [ ]:
adoption_speed_config = OmegaConf.load(
    f"../input/petfinder-pawpularity-score-exp-{ADOPTION_SPEED_EXP_NUM}/exp_{ADOPTION_SPEED_EXP_NUM}.yaml"
)
adoption_speed_config.val_loader.batch_size = 128
pprint(adoption_speed_config)

In [ ]:
gender_config = OmegaConf.load(
    f"../input/petfinder-pawpularity-score-exp-{GENDER_EXP_NUM}/exp_{GENDER_EXP_NUM}.yaml"
)
gender_config.val_loader.batch_size = 128
pprint(gender_config)

In [ ]:
maturity_size_config = OmegaConf.load(
    f"../input/petfinder-pawpularity-score-exp-{MATURITY_SIZE_EXP_NUM}/exp_{MATURITY_SIZE_EXP_NUM}.yaml"
)
maturity_size_config.val_loader.batch_size = 128
pprint(maturity_size_config)

In [ ]:
bin_swin_config = OmegaConf.load(
    f"../input/petfinder-pawpularity-score-exp-{BIN_SWIN_EXP_NUM}/exp_{BIN_SWIN_EXP_NUM}.yaml"
)
bin_swin_config.val_loader.batch_size = 128
pprint(bin_swin_config)

In [ ]:
center_crop_swin_config = OmegaConf.load(
    f"../input/petfinder-pawpularity-score-exp-{CENTER_CROP_SWIN_EXP_NUM}/exp_{CENTER_CROP_SWIN_EXP_NUM}.yaml"
)
center_crop_swin_config.val_loader.batch_size = 128
pprint(center_crop_swin_config)

In [ ]:
seed_everything(paw_config.seed)
torch.autograd.set_detect_anomaly(True)

In [ ]:
test_df = pd.read_csv(Path("../input/petfinder-pawpularity-score")/ "test.csv")
# test_df = pd.concat([test_df]*900, axis="index").reset_index(drop=True)
test_df["org_Id"] = test_df["Id"].copy()
test_df["filepath"] = ("/kaggle/working/yolox_x/test_images/vis_res/test_images/" + test_df["Id"] + ".jpg")
test_df["Id"] = ("/kaggle/working/yolox_x/test_images/vis_res/test_images/" + test_df["Id"] + ".jpg")
test_df["center_Id"] = ("/kaggle/working/test_center_crop/" + test_df["org_Id"] + ".jpg")
display(test_df.head())

In [ ]:
test_df.shape

In [ ]:
# Pawpularity
class PetfinderDataset(Dataset):
    def __init__(self, df, image_size=224, output_dim=1, over_100_or_not=False):
        self._X = df["Id"].values
        self._y = None
        if "Pawpularity" in df.keys():
            pawpularity = df["Pawpularity"].values

            if output_dim == 100:
                self._y = np.zeros((len(df), output_dim)).astype(np.float32)
                for i in range(len(df)):
                    self._y[i, : pawpularity[i]] = 1
            else:
                if over_100_or_not:
                    self._y = (pawpularity == 100).astype(np.float32)
                else:
                    self._y = pawpularity

        self._transform = T.Resize([image_size, image_size])

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path)
        image = self._transform(image)
        if self._y is not None:
            label = self._y[idx]
            return image, label
        return image, 0

class PetfinderInferenceDataModule(LightningDataModule):
    def __init__(
        self,
        test_df,
        cfg,
    ):
        super().__init__()
        self._test_df = test_df
        self._cfg = cfg

    def predict_dataloader(self):
        dataset = PetfinderDataset(
            self._test_df,
            self._cfg.transform.image_size,
            self._cfg.model.output_dim,
            self._cfg.model.over_100_or_not,
        )
        return DataLoader(dataset, **self._cfg.val_loader)
    
# centercrop Pawpularity
class PetfinderCenterDataset(Dataset):
    def __init__(self, df, image_size=224, output_dim=1, over_100_or_not=False):
        self._X = df["center_Id"].values
        self._y = None
        if "Pawpularity" in df.keys():
            pawpularity = df["Pawpularity"].values

            if output_dim == 100:
                self._y = np.zeros((len(df), output_dim)).astype(np.float32)
                for i in range(len(df)):
                    self._y[i, : pawpularity[i]] = 1
            else:
                if over_100_or_not:
                    self._y = (pawpularity == 100).astype(np.float32)
                else:
                    self._y = pawpularity

        self._transform = T.Resize([image_size, image_size])

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path)
        image = self._transform(image)
        if self._y is not None:
            label = self._y[idx]
            return image, label
        return image, 0

class PetfinderCenterInferenceDataModule(LightningDataModule):
    def __init__(
        self,
        test_df,
        cfg,
    ):
        super().__init__()
        self._test_df = test_df
        self._cfg = cfg

    def predict_dataloader(self):
        dataset = PetfinderCenterDataset(
            self._test_df,
            self._cfg.transform.image_size,
            self._cfg.model.output_dim,
            self._cfg.model.over_100_or_not,
        )
        return DataLoader(dataset, **self._cfg.val_loader)


# Age
class PetfinderAgeDataset(Dataset):
    def __init__(self, df, image_size=224, output_dim=1):
        self._X = df["filepath"].values
        self._y = None
        if "Age" in df.keys():
            self._y = df["Age"].clip(0, 100).values

        self._transform = T.Resize([image_size, image_size])

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path, mode=ImageReadMode.RGB)
        image = self._transform(image)
        if self._y is not None:
            label = self._y[idx]
            return image, label
        return image, 0

class PetfinderAgeInferenceDataModule(LightningDataModule):
    def __init__(
        self,
        test_df,
        cfg,
    ):
        super().__init__()
        self._test_df = test_df
        self._cfg = cfg

    def predict_dataloader(self):
        dataset = PetfinderAgeDataset(
            self._test_df,
            self._cfg.transform.image_size,
            self._cfg.model.output_dim,
        )
        return DataLoader(dataset, **self._cfg.val_loader)

# Breed

class PetfinderBreedDataset(Dataset):
    def __init__(self, df, image_size=224, output_dim=100):
        self._X = df["filepath"].values
        self._y = None
        if "Breed1" in df.keys():
            self._y = np.identity(output_dim)[df["Breed1"].values]

        self._transform = T.Resize([image_size, image_size])

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path, mode=ImageReadMode.RGB)
        image = self._transform(image)
        if self._y is not None:
            label = self._y[idx]
            return image, label
        return image, 0

class PetfinderBreedInferenceDataModule(LightningDataModule):
    def __init__(
        self,
        test_df,
        cfg,
    ):
        super().__init__()
        self._test_df = test_df
        self._cfg = cfg

    def predict_dataloader(self):
        dataset = PetfinderBreedDataset(
            self._test_df,
            self._cfg.transform.image_size,
            self._cfg.model.output_dim,
        )
        return DataLoader(dataset, **self._cfg.val_loader)


# Gender
class PetfinderGenderDataset(Dataset):
    def __init__(self, df, image_size=224, output_dim=3):
        self._X = df["filepath"].values
        self._y = None
        if "Gender" in df.keys():
            self._y = np.identity(output_dim)[df["Gender"].values]

        self._transform = T.Resize([image_size, image_size])

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path, mode=ImageReadMode.RGB)
        image = self._transform(image)
        if self._y is not None:
            label = self._y[idx]
            return image, label
        return image, 0


class PetfinderGenderInferenceDataModule(LightningDataModule):
    def __init__(
        self,
        test_df,
        cfg,
    ):
        super().__init__()
        self._test_df = test_df
        self._cfg = cfg

    def predict_dataloader(self):
        dataset = PetfinderGenderDataset(self._test_df, self._cfg.transform.image_size)
        return DataLoader(dataset, **self._cfg.val_loader)

    
# Adoption Speed
class PetfinderAdoptionSpeedDataset(Dataset):
    def __init__(self, df, image_size=224):
        self._X = df["filepath"].values
        self._y = None
        if "AdoptionSpeed" in df.keys():
            adoption_speed = df["AdoptionSpeed"].values

            self._y = np.zeros((len(df), 4)).astype(np.float32)
            for i in range(len(df)):
                self._y[i, : adoption_speed[i]] = 1

        self._transform = T.Resize([image_size, image_size])

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path)
        image = self._transform(image)
        if self._y is not None:
            label = self._y[idx]
            return image, label
        return image, 0


class PetfinderAdoptionSpeedInferenceDataModule(LightningDataModule):
    def __init__(
        self,
        test_df,
        cfg,
    ):
        super().__init__()
        self._test_df = test_df
        self._cfg = cfg

    def predict_dataloader(self):
        dataset = PetfinderAdoptionSpeedDataset(
            self._test_df,
            self._cfg.transform.image_size,
        )
        return DataLoader(dataset, **self._cfg.val_loader)

# bin paw
class PetfinderBinPawpularityDataset(Dataset):
    def __init__(self, df, image_size=224, num_bins=14):
        self._X = df["filepath"].values
        self._y = None
        if "bins_paw" in df.keys():
            adoption_speed = df["bins_paw"].values

            self._y = np.zeros((len(df), num_bins)).astype(np.float32)
            for i in range(len(df)):
                self._y[i, : adoption_speed[i]] = 1

        self._transform = T.Resize([image_size, image_size])

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path)
        image = self._transform(image)
        if self._y is not None:
            label = self._y[idx]
            return image, label
        return image, 0

class PetfinderBinPawpularityInferenceDataModule(LightningDataModule):
    def __init__(
        self,
        test_df,
        cfg,
    ):
        super().__init__()
        self._test_df = test_df
        self._cfg = cfg

    def predict_dataloader(self):
        dataset = PetfinderBinPawpularityDataset(
            self._test_df,
            self._cfg.transform.image_size,
        )
        return DataLoader(dataset, **self._cfg.val_loader)
    
    
# Maturity Size
class PetfinderMaturitySizeDataset(Dataset):
    def __init__(self, df, image_size=224):
        self._X = df["filepath"].values
        self._y = None
        if "MaturitySize" in df.keys():
            maturity_size = df["MaturitySize"].values

            self._y = np.zeros((len(df), 3)).astype(np.float32)
            for i in range(len(df)):
                self._y[i, : maturity_size[i]] = 1

        self._transform = T.Resize([image_size, image_size])

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path)
        image = self._transform(image)
        if self._y is not None:
            label = self._y[idx]
            return image, label
        return image, 0

class PetfinderMaturitySizeInferenceDataModule(LightningDataModule):
    def __init__(
        self,
        test_df,
        cfg,
    ):
        super().__init__()
        self._test_df = test_df
        self._cfg = cfg

    def predict_dataloader(self):
        dataset = PetfinderMaturitySizeDataset(
            self._test_df,
            self._cfg.transform.image_size,
        )
        return DataLoader(dataset, **self._cfg.val_loader)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=True, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss


In [ ]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB

def get_default_transforms():
    transform = {
        "train": T.Compose(
            [
                T.RandomHorizontalFlip(),
                T.RandomVerticalFlip(),
                T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
        ),
        "val": T.Compose(
            [
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
        ),
    }
    return transform


In [ ]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1.0 - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2


def rand_region(target_size, source_size):
    t_h, t_w = target_size[2:]
    s_h, s_w = source_size[2:]
    cut_h = s_h // 2
    cut_w = s_w // 2

    cx = np.random.randint(cut_w, t_w - cut_w)
    cy = np.random.randint(cut_h, t_h - cut_h)
    x1 = cx - cut_w
    x2 = x1 + s_w
    y1 = cy - cut_h
    y2 = y1 + s_h
    return x1, y1, x2, y2

import numpy as np
import torch
from torchvision.transforms import Resize

def cutmix(x, y, alpha):
    assert alpha > 0, "alpha should be larger than 0"
    lam = np.random.beta(alpha, alpha)
    rand_index = torch.randperm(x.size()[0]).cuda()
    target_a = y
    target_b = y[rand_index]
    bbx1, bby1, bbx2, bby2 = rand_bbox(x.size(), lam)
    x[:, :, bbx1:bbx2, bby1:bby2] = x[rand_index, :, bbx1:bbx2, bby1:bby2]
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.size()[-1] * x.size()[-2]))
    return x, target_a, target_b, lam


def mixup(x, y, alpha):
    assert alpha > 0, "alpha should be larger than 0"

    lam = np.random.beta(alpha, alpha)
    rand_index = torch.randperm(x.size()[0]).cuda()
    mixed_x = lam * x + (1 - lam) * x[rand_index, :]
    target_a, target_b = y, y[rand_index]
    return mixed_x, target_a, target_b, lam


def resizemix(x, y, alpha=0.1, beta=0.8):
    assert alpha > 0, "alpha should be larger than 0"
    assert beta < 1, "beta should be smaller than 1"

    rand_index = torch.randperm(x.size()[0]).cuda()
    tau = np.random.uniform(alpha, beta)
    lam = tau ** 2

    H, W = x.size()[2:]
    resize_transform = Resize((int(H * tau), int(W * tau)))
    resized_x = resize_transform(x[rand_index])

    target_a = y[rand_index]
    target_b = y
    x1, y1, x2, y2 = rand_region(x.size(), resized_x.size())
    x[:, :, y1:y2, x1:x2] = resized_x
    return x, target_a, target_b, lam


def get_strong_transforms(cfg):
    return eval(cfg.strong_transform.name)

In [ ]:
# Copyright 2021 Sea Limited.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
Vision OutLOoker (VOLO) implementation
"""

# https://github.com/sail-sg/volo/blob/main/models/volo.py

import math

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.models.helpers import load_state_dict
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
from timm.models.registry import register_model


def _cfg(url="", **kwargs):
    return {
        "url": url,
        "num_classes": 1000,
        "input_size": (3, 224, 224),
        "pool_size": None,
        "crop_pct": 0.96,
        "interpolation": "bicubic",
        "mean": IMAGENET_DEFAULT_MEAN,
        "std": IMAGENET_DEFAULT_STD,
        "first_conv": "patch_embed.proj",
        "classifier": "head",
        **kwargs,
    }


default_cfgs = {
    "volo": _cfg(crop_pct=0.96),
    "volo_large": _cfg(crop_pct=1.15),
}


class OutlookAttention(nn.Module):
    """
    Implementation of outlook attention
    --dim: hidden dim
    --num_heads: number of heads
    --kernel_size: kernel size in each window for outlook attention
    return: token features after outlook attention
    """

    def __init__(
        self,
        dim,
        num_heads,
        kernel_size=3,
        padding=1,
        stride=1,
        qkv_bias=False,
        qk_scale=None,
        attn_drop=0.0,
        proj_drop=0.0,
    ):
        super().__init__()
        head_dim = dim // num_heads
        self.num_heads = num_heads
        self.kernel_size = kernel_size
        self.padding = padding
        self.stride = stride
        self.scale = qk_scale or head_dim ** -0.5

        self.v = nn.Linear(dim, dim, bias=qkv_bias)
        self.attn = nn.Linear(dim, kernel_size ** 4 * num_heads)

        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        self.unfold = nn.Unfold(kernel_size=kernel_size, padding=padding, stride=stride)
        self.pool = nn.AvgPool2d(kernel_size=stride, stride=stride, ceil_mode=True)

    def forward(self, x):
        B, H, W, C = x.shape

        v = self.v(x).permute(0, 3, 1, 2)  # B, C, H, W

        h, w = math.ceil(H / self.stride), math.ceil(W / self.stride)
        v = (
            self.unfold(v)
            .reshape(
                B,
                self.num_heads,
                C // self.num_heads,
                self.kernel_size * self.kernel_size,
                h * w,
            )
            .permute(0, 1, 4, 3, 2)
        )  # B,H,N,kxk,C/H

        attn = self.pool(x.permute(0, 3, 1, 2)).permute(0, 2, 3, 1)
        attn = (
            self.attn(attn)
            .reshape(
                B,
                h * w,
                self.num_heads,
                self.kernel_size * self.kernel_size,
                self.kernel_size * self.kernel_size,
            )
            .permute(0, 2, 1, 3, 4)
        )  # B,H,N,kxk,kxk
        attn = attn * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (
            (attn @ v)
            .permute(0, 1, 4, 3, 2)
            .reshape(B, C * self.kernel_size * self.kernel_size, h * w)
        )
        x = F.fold(
            x,
            output_size=(H, W),
            kernel_size=self.kernel_size,
            padding=self.padding,
            stride=self.stride,
        )

        x = self.proj(x.permute(0, 2, 3, 1))
        x = self.proj_drop(x)

        return x


class Outlooker(nn.Module):
    """
    Implementation of outlooker layer: which includes outlook attention + MLP
    Outlooker is the first stage in our VOLO
    --dim: hidden dim
    --num_heads: number of heads
    --mlp_ratio: mlp ratio
    --kernel_size: kernel size in each window for outlook attention
    return: outlooker layer
    """

    def __init__(
        self,
        dim,
        kernel_size,
        padding,
        stride=1,
        num_heads=1,
        mlp_ratio=3.0,
        attn_drop=0.0,
        drop_path=0.0,
        act_layer=nn.GELU,
        norm_layer=nn.LayerNorm,
        qkv_bias=False,
        qk_scale=None,
    ):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = OutlookAttention(
            dim,
            num_heads,
            kernel_size=kernel_size,
            padding=padding,
            stride=stride,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
        )

        self.drop_path = DropPath(drop_path) if drop_path > 0.0 else nn.Identity()

        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(
            in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer
        )

    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x


class Mlp(nn.Module):
    "Implementation of MLP"

    def __init__(
        self,
        in_features,
        hidden_features=None,
        out_features=None,
        act_layer=nn.GELU,
        drop=0.0,
    ):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class Attention(nn.Module):
    "Implementation of self-attention"

    def __init__(
        self,
        dim,
        num_heads=8,
        qkv_bias=False,
        qk_scale=None,
        attn_drop=0.0,
        proj_drop=0.0,
    ):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, H, W, C = x.shape

        qkv = (
            self.qkv(x)
            .reshape(B, H * W, 3, self.num_heads, C // self.num_heads)
            .permute(2, 0, 3, 1, 4)
        )
        q, k, v = (
            qkv[0],
            qkv[1],
            qkv[2],
        )  # make torchscript happy (cannot use tensor as tuple)

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, H, W, C)
        x = self.proj(x)
        x = self.proj_drop(x)

        return x


class Transformer(nn.Module):
    """
    Implementation of Transformer,
    Transformer is the second stage in our VOLO
    """

    def __init__(
        self,
        dim,
        num_heads,
        mlp_ratio=4.0,
        qkv_bias=False,
        qk_scale=None,
        attn_drop=0.0,
        drop_path=0.0,
        act_layer=nn.GELU,
        norm_layer=nn.LayerNorm,
    ):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(
            dim,
            num_heads=num_heads,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
        )

        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_path = DropPath(drop_path) if drop_path > 0.0 else nn.Identity()

        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(
            in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer
        )

    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x


class ClassAttention(nn.Module):
    """
    Class attention layer from CaiT, see details in CaiT
    Class attention is the post stage in our VOLO, which is optional.
    """

    def __init__(
        self,
        dim,
        num_heads=8,
        head_dim=None,
        qkv_bias=False,
        qk_scale=None,
        attn_drop=0.0,
        proj_drop=0.0,
    ):
        super().__init__()
        self.num_heads = num_heads
        if head_dim is not None:
            self.head_dim = head_dim
        else:
            head_dim = dim // num_heads
            self.head_dim = head_dim
        self.scale = qk_scale or head_dim ** -0.5

        self.kv = nn.Linear(dim, self.head_dim * self.num_heads * 2, bias=qkv_bias)
        self.q = nn.Linear(dim, self.head_dim * self.num_heads, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(self.head_dim * self.num_heads, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape

        kv = (
            self.kv(x)
            .reshape(B, N, 2, self.num_heads, self.head_dim)
            .permute(2, 0, 3, 1, 4)
        )
        k, v = kv[0], kv[1]  # make torchscript happy (cannot use tensor as tuple)
        q = self.q(x[:, :1, :]).reshape(B, self.num_heads, 1, self.head_dim)
        attn = (q * self.scale) @ k.transpose(-2, -1)
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        cls_embed = (
            (attn @ v).transpose(1, 2).reshape(B, 1, self.head_dim * self.num_heads)
        )
        cls_embed = self.proj(cls_embed)
        cls_embed = self.proj_drop(cls_embed)
        return cls_embed


class ClassBlock(nn.Module):
    """
    Class attention block from CaiT, see details in CaiT
    We use two-layers class attention in our VOLO, which is optional.
    """

    def __init__(
        self,
        dim,
        num_heads,
        head_dim=None,
        mlp_ratio=4.0,
        qkv_bias=False,
        qk_scale=None,
        drop=0.0,
        attn_drop=0.0,
        drop_path=0.0,
        act_layer=nn.GELU,
        norm_layer=nn.LayerNorm,
    ):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = ClassAttention(
            dim,
            num_heads=num_heads,
            head_dim=head_dim,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
            proj_drop=drop,
        )
        # NOTE: drop path for stochastic depth
        self.drop_path = DropPath(drop_path) if drop_path > 0.0 else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(
            in_features=dim,
            hidden_features=mlp_hidden_dim,
            act_layer=act_layer,
            drop=drop,
        )

    def forward(self, x):
        cls_embed = x[:, :1]
        cls_embed = cls_embed + self.drop_path(self.attn(self.norm1(x)))
        cls_embed = cls_embed + self.drop_path(self.mlp(self.norm2(cls_embed)))
        return torch.cat([cls_embed, x[:, 1:]], dim=1)


def get_block(block_type, **kargs):
    """
    get block by name, specifically for class attention block in here
    """
    if block_type == "ca":
        return ClassBlock(**kargs)


def rand_bbox(size, lam, scale=1):
    """
    get bounding box as token labeling (https://github.com/zihangJiang/TokenLabeling)
    return: bounding box
    """
    W = size[1] // scale
    H = size[2] // scale
    cut_rat = np.sqrt(1.0 - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2


class PatchEmbed(nn.Module):
    """
    Image to Patch Embedding.
    Different with ViT use 1 conv layer, we use 4 conv layers to do patch embedding
    """

    def __init__(
        self,
        img_size=224,
        stem_conv=False,
        stem_stride=1,
        patch_size=8,
        in_chans=3,
        hidden_dim=64,
        embed_dim=384,
    ):
        super().__init__()
        assert patch_size in [4, 8, 16]

        self.stem_conv = stem_conv
        if stem_conv:
            self.conv = nn.Sequential(
                nn.Conv2d(
                    in_chans,
                    hidden_dim,
                    kernel_size=7,
                    stride=stem_stride,
                    padding=3,
                    bias=False,
                ),  # 112x112
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU(inplace=True),
                nn.Conv2d(
                    hidden_dim,
                    hidden_dim,
                    kernel_size=3,
                    stride=1,
                    padding=1,
                    bias=False,
                ),  # 112x112
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU(inplace=True),
                nn.Conv2d(
                    hidden_dim,
                    hidden_dim,
                    kernel_size=3,
                    stride=1,
                    padding=1,
                    bias=False,
                ),  # 112x112
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU(inplace=True),
            )

        self.proj = nn.Conv2d(
            hidden_dim,
            embed_dim,
            kernel_size=patch_size // stem_stride,
            stride=patch_size // stem_stride,
        )
        self.num_patches = (img_size // patch_size) * (img_size // patch_size)

    def forward(self, x):
        if self.stem_conv:
            x = self.conv(x)
        x = self.proj(x)  # B, C, H, W
        return x


class Downsample(nn.Module):
    """
    Image to Patch Embedding, downsampling between stage1 and stage2
    """

    def __init__(self, in_embed_dim, out_embed_dim, patch_size):
        super().__init__()
        self.proj = nn.Conv2d(
            in_embed_dim, out_embed_dim, kernel_size=patch_size, stride=patch_size
        )

    def forward(self, x):
        x = x.permute(0, 3, 1, 2)
        x = self.proj(x)  # B, C, H, W
        x = x.permute(0, 2, 3, 1)
        return x


def outlooker_blocks(
    block_fn,
    index,
    dim,
    layers,
    num_heads=1,
    kernel_size=3,
    padding=1,
    stride=1,
    mlp_ratio=3.0,
    qkv_bias=False,
    qk_scale=None,
    attn_drop=0,
    drop_path_rate=0.0,
    **kwargs,
):
    """
    generate outlooker layer in stage1
    return: outlooker layers
    """
    blocks = []
    for block_idx in range(layers[index]):
        block_dpr = (
            drop_path_rate * (block_idx + sum(layers[:index])) / (sum(layers) - 1)
        )
        blocks.append(
            block_fn(
                dim,
                kernel_size=kernel_size,
                padding=padding,
                stride=stride,
                num_heads=num_heads,
                mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias,
                qk_scale=qk_scale,
                attn_drop=attn_drop,
                drop_path=block_dpr,
            )
        )

    blocks = nn.Sequential(*blocks)

    return blocks


def transformer_blocks(
    block_fn,
    index,
    dim,
    layers,
    num_heads,
    mlp_ratio=3.0,
    qkv_bias=False,
    qk_scale=None,
    attn_drop=0,
    drop_path_rate=0.0,
    **kwargs,
):
    """
    generate transformer layers in stage2
    return: transformer layers
    """
    blocks = []
    for block_idx in range(layers[index]):
        block_dpr = (
            drop_path_rate * (block_idx + sum(layers[:index])) / (sum(layers) - 1)
        )
        blocks.append(
            block_fn(
                dim,
                num_heads,
                mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias,
                qk_scale=qk_scale,
                attn_drop=attn_drop,
                drop_path=block_dpr,
            )
        )

    blocks = nn.Sequential(*blocks)

    return blocks


class VOLO(nn.Module):
    """
    Vision Outlooker, the main class of our model
    --layers: [x,x,x,x], four blocks in two stages, the first block is outlooker, the
              other three are transformer, we set four blocks, which are easily
              applied to downstream tasks
    --img_size, --in_chans, --num_classes: these three are very easy to understand
    --patch_size: patch_size in outlook attention
    --stem_hidden_dim: hidden dim of patch embedding, d1-d4 is 64, d5 is 128
    --embed_dims, --num_heads: embedding dim, number of heads in each block
    --downsamples: flags to apply downsampling or not
    --outlook_attention: flags to apply outlook attention or not
    --mlp_ratios, --qkv_bias, --qk_scale, --drop_rate: easy to undertand
    --attn_drop_rate, --drop_path_rate, --norm_layer: easy to undertand
    --post_layers: post layers like two class attention layers using [ca, ca],
                  if yes, return_mean=False
    --return_mean: use mean of all feature tokens for classification, if yes, no class token
    --return_dense: use token labeling, details are here:
                    https://github.com/zihangJiang/TokenLabeling
    --mix_token: mixing tokens as token labeling, details are here:
                    https://github.com/zihangJiang/TokenLabeling
    --pooling_scale: pooling_scale=2 means we downsample 2x
    --out_kernel, --out_stride, --out_padding: kerner size,
                                               stride, and padding for outlook attention
    """

    def __init__(
        self,
        layers,
        img_size=224,
        in_chans=3,
        num_classes=1000,
        patch_size=8,
        stem_hidden_dim=64,
        embed_dims=None,
        num_heads=None,
        downsamples=None,
        outlook_attention=None,
        mlp_ratios=None,
        qkv_bias=False,
        qk_scale=None,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.0,
        norm_layer=nn.LayerNorm,
        post_layers=None,
        return_mean=False,
        return_dense=True,
        mix_token=False,
        pooling_scale=2,
        out_kernel=3,
        out_stride=2,
        out_padding=1,
    ):

        super().__init__()
        self.num_classes = num_classes
        self.num_features = embed_dims[-1]
        self.patch_embed = PatchEmbed(
            stem_conv=True,
            stem_stride=2,
            patch_size=patch_size,
            in_chans=in_chans,
            hidden_dim=stem_hidden_dim,
            embed_dim=embed_dims[0],
        )

        # inital positional encoding, we add positional encoding after outlooker blocks
        self.pos_embed = nn.Parameter(
            torch.zeros(
                1,
                img_size // patch_size // pooling_scale,
                img_size // patch_size // pooling_scale,
                embed_dims[-1],
            )
        )

        self.pos_drop = nn.Dropout(p=drop_rate)

        # set the main block in network
        network = []
        for i in range(len(layers)):
            if outlook_attention[i]:
                # stage 1
                stage = outlooker_blocks(
                    Outlooker,
                    i,
                    embed_dims[i],
                    layers,
                    downsample=downsamples[i],
                    num_heads=num_heads[i],
                    kernel_size=out_kernel,
                    stride=out_stride,
                    padding=out_padding,
                    mlp_ratio=mlp_ratios[i],
                    qkv_bias=qkv_bias,
                    qk_scale=qk_scale,
                    attn_drop=attn_drop_rate,
                    norm_layer=norm_layer,
                )
                network.append(stage)
            else:
                # stage 2
                stage = transformer_blocks(
                    Transformer,
                    i,
                    embed_dims[i],
                    layers,
                    num_heads[i],
                    mlp_ratio=mlp_ratios[i],
                    qkv_bias=qkv_bias,
                    qk_scale=qk_scale,
                    drop_path_rate=drop_path_rate,
                    attn_drop=attn_drop_rate,
                    norm_layer=norm_layer,
                )
                network.append(stage)

            if downsamples[i]:
                # downsampling between two stages
                network.append(Downsample(embed_dims[i], embed_dims[i + 1], 2))

        self.network = nn.ModuleList(network)

        # set post block, for example, class attention layers
        self.post_network = None
        if post_layers is not None:
            self.post_network = nn.ModuleList(
                [
                    get_block(
                        post_layers[i],
                        dim=embed_dims[-1],
                        num_heads=num_heads[-1],
                        mlp_ratio=mlp_ratios[-1],
                        qkv_bias=qkv_bias,
                        qk_scale=qk_scale,
                        attn_drop=attn_drop_rate,
                        drop_path=0.0,
                        norm_layer=norm_layer,
                    )
                    for i in range(len(post_layers))
                ]
            )
            self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dims[-1]))
            trunc_normal_(self.cls_token, std=0.02)

        # set output type
        self.return_mean = return_mean  # if yes, return mean, not use class token
        self.return_dense = (
            return_dense  # if yes, return class token and all feature tokens
        )
        if return_dense:
            assert not return_mean, "cannot return both mean and dense"
        self.mix_token = mix_token
        self.pooling_scale = pooling_scale
        if mix_token:  # enable token mixing, see token labeling for details.
            self.beta = 1.0
            assert return_dense, "return all tokens if mix_token is enabled"
        if return_dense:
            self.aux_head = (
                nn.Linear(embed_dims[-1], num_classes)
                if num_classes > 0
                else nn.Identity()
            )
        self.norm = norm_layer(embed_dims[-1])

        # Classifier head
        self.head = (
            nn.Linear(embed_dims[-1], num_classes) if num_classes > 0 else nn.Identity()
        )

        trunc_normal_(self.pos_embed, std=0.02)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=0.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @torch.jit.ignore
    def no_weight_decay(self):
        return {"pos_embed", "cls_token"}

    def get_classifier(self):
        return self.head

    def reset_classifier(self, num_classes):
        self.num_classes = num_classes
        self.head = (
            nn.Linear(self.embed_dim, num_classes) if num_classes > 0 else nn.Identity()
        )

    def forward_embeddings(self, x):
        # patch embedding
        x = self.patch_embed(x)
        # B,C,H,W-> B,H,W,C
        x = x.permute(0, 2, 3, 1)
        return x

    def forward_tokens(self, x):
        for idx, block in enumerate(self.network):
            if idx == 2:  # add positional encoding after outlooker blocks
                x = x + self.pos_embed
                x = self.pos_drop(x)
            x = block(x)

        B, H, W, C = x.shape
        x = x.reshape(B, -1, C)
        return x

    def forward_cls(self, x):
        B, N, C = x.shape
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        for block in self.post_network:
            x = block(x)
        return x

    def forward(self, x):
        # step1: patch embedding
        x = self.forward_embeddings(x)

        # mix token, see token labeling for details.
        if self.mix_token and self.training:
            lam = np.random.beta(self.beta, self.beta)
            patch_h, patch_w = (
                x.shape[1] // self.pooling_scale,
                x.shape[2] // self.pooling_scale,
            )
            bbx1, bby1, bbx2, bby2 = rand_bbox(x.size(), lam, scale=self.pooling_scale)
            temp_x = x.clone()
            sbbx1, sbby1, sbbx2, sbby2 = (
                self.pooling_scale * bbx1,
                self.pooling_scale * bby1,
                self.pooling_scale * bbx2,
                self.pooling_scale * bby2,
            )
            temp_x[:, sbbx1:sbbx2, sbby1:sbby2, :] = x.flip(0)[
                :, sbbx1:sbbx2, sbby1:sbby2, :
            ]
            x = temp_x
        else:
            bbx1, bby1, bbx2, bby2 = 0, 0, 0, 0

        # step2: tokens learning in the two stages
        x = self.forward_tokens(x)

        # step3: post network, apply class attention or not
        if self.post_network is not None:
            x = self.forward_cls(x)
        x = self.norm(x)

        if self.return_mean:  # if no class token, return mean
            return self.head(x.mean(1))

        x_cls = self.head(x[:, 0])
        if not self.return_dense:
            return x_cls

        x_aux = self.aux_head(
            x[:, 1:]
        )  # generate classes in all feature tokens, see token labeling

        if not self.training:
            return x_cls + 0.5 * x_aux.max(1)[0]

        # if (
        #     self.mix_token and self.training
        # ):  # reverse "mix token", see token labeling for details.
        #     x_aux = x_aux.reshape(x_aux.shape[0], patch_h, patch_w, x_aux.shape[-1])
        #
        #     temp_x = x_aux.clone()
        #     temp_x[:, bbx1:bbx2, bby1:bby2, :] = x_aux.flip(0)[
        #         :, bbx1:bbx2, bby1:bby2, :
        #     ]
        #     x_aux = temp_x
        #
        #     x_aux = x_aux.reshape(x_aux.shape[0], patch_h * patch_w, x_aux.shape[-1])
        #
        # # return these: 1. class token, 2. classes from all feature tokens, 3. bounding box
        # return x_cls, x_aux, (bbx1, bby1, bbx2, bby2)

        return x_cls + 0.5 * x_aux.max(1)[0]


@register_model
def volo_d1(pretrained=False, **kwargs):
    """
    VOLO-D1 model, Params: 27M
    --layers: [x,x,x,x], four blocks in two stages, the first stage(block) is outlooker,
            the other three blocks are transformer, we set four blocks, which are easily
             applied to downstream tasks
    --embed_dims, --num_heads,: embedding dim, number of heads in each block
    --downsamples: flags to apply downsampling or not in four blocks
    --outlook_attention: flags to apply outlook attention or not
    --mlp_ratios: mlp ratio in four blocks
    --post_layers: post layers like two class attention layers using [ca, ca]
    See detail for all args in the class VOLO()
    """
    layers = [4, 4, 8, 2]  # num of layers in the four blocks
    embed_dims = [192, 384, 384, 384]
    num_heads = [6, 12, 12, 12]
    mlp_ratios = [3, 3, 3, 3]
    downsamples = [True, False, False, False]  # do downsampling after first block
    outlook_attention = [True, False, False, False]
    # first block is outlooker (stage1), the other three are transformer (stage2)
    model = VOLO(
        layers,
        embed_dims=embed_dims,
        num_heads=num_heads,
        mlp_ratios=mlp_ratios,
        downsamples=downsamples,
        outlook_attention=outlook_attention,
        post_layers=["ca", "ca"],
        **kwargs,
    )
    model.default_cfg = default_cfgs["volo"]
    return model


@register_model
def volo_d2(pretrained=False, **kwargs):
    """
    VOLO-D2 model, Params: 59M
    """
    layers = [6, 4, 10, 4]
    embed_dims = [256, 512, 512, 512]
    num_heads = [8, 16, 16, 16]
    mlp_ratios = [3, 3, 3, 3]
    downsamples = [True, False, False, False]
    outlook_attention = [True, False, False, False]
    model = VOLO(
        layers,
        embed_dims=embed_dims,
        num_heads=num_heads,
        mlp_ratios=mlp_ratios,
        downsamples=downsamples,
        outlook_attention=outlook_attention,
        post_layers=["ca", "ca"],
        **kwargs,
    )
    model.default_cfg = default_cfgs["volo"]
    return model


@register_model
def volo_d3(pretrained=False, **kwargs):
    """
    VOLO-D3 model, Params: 86M
    """
    layers = [8, 8, 16, 4]
    embed_dims = [256, 512, 512, 512]
    num_heads = [8, 16, 16, 16]
    mlp_ratios = [3, 3, 3, 3]
    downsamples = [True, False, False, False]
    outlook_attention = [True, False, False, False]
    model = VOLO(
        layers,
        embed_dims=embed_dims,
        num_heads=num_heads,
        mlp_ratios=mlp_ratios,
        downsamples=downsamples,
        outlook_attention=outlook_attention,
        post_layers=["ca", "ca"],
        **kwargs,
    )
    model.default_cfg = default_cfgs["volo"]
    return model


@register_model
def volo_d4(pretrained=False, **kwargs):
    """
    VOLO-D4 model, Params: 193M
    """
    layers = [8, 8, 16, 4]
    embed_dims = [384, 768, 768, 768]
    num_heads = [12, 16, 16, 16]
    mlp_ratios = [3, 3, 3, 3]
    downsamples = [True, False, False, False]
    outlook_attention = [True, False, False, False]
    model = VOLO(
        layers,
        embed_dims=embed_dims,
        num_heads=num_heads,
        mlp_ratios=mlp_ratios,
        downsamples=downsamples,
        outlook_attention=outlook_attention,
        post_layers=["ca", "ca"],
        **kwargs,
    )
    model.default_cfg = default_cfgs["volo_large"]
    return model


@register_model
def volo_d5(pretrained=False, **kwargs):
    """
    VOLO-D5 model, Params: 296M
    stem_hidden_dim=128, the dim in patch embedding is 128 for VOLO-D5
    """
    layers = [12, 12, 20, 4]
    embed_dims = [384, 768, 768, 768]
    num_heads = [12, 16, 16, 16]
    mlp_ratios = [4, 4, 4, 4]
    downsamples = [True, False, False, False]
    outlook_attention = [True, False, False, False]
    model = VOLO(
        layers,
        embed_dims=embed_dims,
        num_heads=num_heads,
        mlp_ratios=mlp_ratios,
        downsamples=downsamples,
        outlook_attention=outlook_attention,
        post_layers=["ca", "ca"],
        stem_hidden_dim=128,
        **kwargs,
    )
    model.default_cfg = default_cfgs["volo_large"]
    return model


def volo_load_pretrained_weights(
    model, checkpoint_path, use_ema=False, strict=True, num_classes=1000
):
    """load pretrained weight for VOLO models"""
    state_dict = load_state_dict(checkpoint_path, model, use_ema, num_classes)
    model.load_state_dict(state_dict, strict=strict)


In [ ]:
import torch
import torch.optim as optim
from pytorch_lightning import LightningModule
from timm import create_model
from torch import nn



class Model(LightningModule):
    def __init__(self, cfg, batch_size=32):
        super().__init__()
        self.cfg = cfg
        self.batch_size = batch_size
        self.__build_model()
        self._criterion = eval(self.cfg.loss)()
        self.transform = get_default_transforms()
        self.strong_transform = get_strong_transforms(self.cfg)
        self.save_hyperparameters(cfg)

    def __build_model(self):
        self.backbone = create_model(
            self.cfg.model.name, pretrained=False, num_classes=0, in_chans=3
        )
        num_features = self.backbone.num_features
        if self.cfg.model.name == "CSWin_64_12211_tiny_224":
            self.backbone.load_state_dict(
                torch.hub.load_state_dict_from_url(
                    "https://github.com/microsoft/CSWin-Transformer/releases/download/v0.1.0/cswin_tiny_224.pth"
                ),
                strict=False,
            )
        elif self.cfg.model.name == "volo_d1":
            self.backbone.load_state_dict(
                torch.hub.load_state_dict_from_url(
                    "https://github.com/sail-sg/volo/releases/download/volo_1/d1_224_84.2.pth.tar"
                ),
                strict=False,
            )
        elif self.cfg.model.name == "volo_d5":
            self.backbone.load_state_dict(
                torch.hub.load_state_dict_from_url(
                    "https://github.com/sail-sg/volo/releases/download/volo_1/d5_224_86.10.pth.tar"
                ),
                strict=False,
            )

        self.fc = nn.Sequential(
            nn.Dropout(0.5), nn.Linear(num_features, self.cfg.model.output_dim)
        )

    def forward(self, x):
        f = self.backbone(x)
        out = self.fc(f)
        return out

    def training_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, "train")
        return {"loss": loss, "pred": pred, "labels": labels}

    def validation_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, "val")
        return {"pred": pred, "labels": labels}

    def predict_step(self, batch, batch_idx, dataloader_idx=None):
        images, labels = batch
        images = self.transform["val"](images)
        embed = self.backbone(images).detach().cpu()
        out = self.forward(images).squeeze(1)
        if self.cfg.model.output_dim == 1:
            out = out.sigmoid().detach().cpu() * 100.0
        else:
            pred = out.sigmoid().detach().cpu().sum(axis=1)
            out = (pred, out.sigmoid().detach().cpu())
        return out, embed

    def __share_step(self, batch, mode):
        images, labels = batch
        labels = labels.float()
        if self.cfg.model.output_dim == 1:
            labels /= 100.0

        images = self.transform[mode](images)

        if torch.rand(1)[0] < 0.5 and mode == "train":
            mix_images, target_a, target_b, lam = self.strong_transform(
                images, labels, **self.cfg.strong_transform.params
            )
            logits = self.forward(mix_images).squeeze(1)
            loss = self._criterion(logits, target_a) * lam + (
                1 - lam
            ) * self._criterion(logits, target_b)
        else:
            logits = self.forward(images).squeeze(1)
            loss = self._criterion(logits, labels)

        if self.cfg.model.output_dim == 1:
            pred = logits.sigmoid().detach().cpu() * 100.0
            labels = labels.detach().cpu() * 100.0
        else:
            pred = logits.sigmoid().detach().cpu().sum(axis=1)
            labels = labels.detach().cpu().sum(axis=1)
        return loss, pred, labels

    def training_epoch_end(self, outputs):
        self.__share_epoch_end(outputs, "train")

    def validation_epoch_end(self, outputs):
        self.__share_epoch_end(outputs, "val")

    def __share_epoch_end(self, outputs, mode):
        preds = []
        labels = []
        for out in outputs:
            pred, label = out["pred"], out["labels"]
            preds.append(pred)
            labels.append(label)
        preds = torch.cat(preds)
        labels = torch.cat(labels)
        metrics = torch.sqrt(((labels - preds) ** 2).mean())
        self.log(f"{mode}_loss", metrics)

    def check_gradcam(
        self, dataloader, target_layer, target_category, reshape_transform=None
    ):
        cam = GradCAMPlusPlus(
            model=self,
            target_layer=target_layer,
            use_cuda=self.cfg.trainer.gpus,
            reshape_transform=reshape_transform,
        )

        org_images, labels = iter(dataloader).next()
        cam.batch_size = len(org_images)
        images = self.transform["val"](org_images)
        images = images.to(self.device)
        logits = self.forward(images).squeeze(1)
        pred = logits.sigmoid().detach().cpu().numpy() * 100
        labels = labels.cpu().numpy()

        grayscale_cam = cam(
            input_tensor=images, target_category=target_category, eigen_smooth=True
        )
        org_images = org_images.detach().cpu().numpy().transpose(0, 2, 3, 1) / 255.0
        return org_images, grayscale_cam, pred, labels

    def configure_optimizers(self):
        optimizer = eval(self.cfg.optimizer.name)(
            self.parameters(), **self.cfg.optimizer.params
        )
        scheduler = eval(self.cfg.scheduler.name)(
            optimizer, **self.cfg.scheduler.params
        )
        return [optimizer], [scheduler]

    def optimizer_zero_grad(self, epoch, batch_idx, optimizer, optimizer_idx):
        # https://pytorch-lightning.readthedocs.io/en/latest/guides/speed.html#set-grads-to-none
        optimizer.zero_grad(set_to_none=False)


class ClassificationModel(LightningModule):
    def __init__(self, cfg, batch_size=32):
        super().__init__()
        self.cfg = cfg
        self.batch_size = batch_size
        self.__build_model()
        self._criterion = eval(self.cfg.loss)()
        self.transform = get_default_transforms()
        self.strong_transform = get_strong_transforms(self.cfg)
        self.save_hyperparameters(cfg)

    def __build_model(self):
        self.backbone = create_model(
            self.cfg.model.name, pretrained=False, num_classes=0, in_chans=3
        )
        num_features = self.backbone.num_features
        self.fc = nn.Sequential(
            nn.Dropout(0.5), nn.Linear(num_features, self.cfg.model.output_dim)
        )

    def forward(self, x):
        f = self.backbone(x)
        out = self.fc(f)
        return out

    def training_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, "train")
        return {"loss": loss, "pred": pred, "labels": labels}

    def validation_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, "val")
        return {"pred": pred, "labels": labels}

    def predict_step(self, batch, batch_idx, dataloader_idx=None):
        images, labels = batch
        images = self.transform["val"](images)
        out = self.forward(images).squeeze(1)
        out = out.sigmoid().detach().cpu()
        return out

    def __share_step(self, batch, mode):
        images, labels = batch
        labels = labels.float()

        images = self.transform[mode](images)

        if torch.rand(1)[0] < 0.5 and mode == "train":
            mix_images, target_a, target_b, lam = self.strong_transform(
                images, labels, **self.cfg.strong_transform.params
            )
            logits = self.forward(mix_images).squeeze(1)
            loss = self._criterion(logits, target_a) * lam + (
                1 - lam
            ) * self._criterion(logits, target_b)
        else:
            logits = self.forward(images).squeeze(1)
            loss = self._criterion(logits, labels)

        pred = logits.sigmoid().detach().cpu()
        labels = labels.detach().cpu()
        return loss, pred, labels

    def training_epoch_end(self, outputs):
        self.__share_epoch_end(outputs, "train")

    def validation_epoch_end(self, outputs):
        self.__share_epoch_end(outputs, "val")

    def __share_epoch_end(self, outputs, mode):
        preds = []
        labels = []
        for out in outputs:
            pred, label = out["pred"], out["labels"]
            preds.append(pred)
            labels.append(label)
        preds = torch.cat(preds)
        labels = torch.cat(labels)
        metrics = torch.nn.BCEWithLogitsLoss()(
            preds.to(torch.float32), labels.to(torch.float32)
        )
        self.log(f"{mode}_loss", metrics)

    def check_gradcam(
        self, dataloader, target_layer, target_category, reshape_transform=None
    ):
        cam = GradCAMPlusPlus(
            model=self,
            target_layer=target_layer,
            use_cuda=self.cfg.trainer.gpus,
            reshape_transform=reshape_transform,
        )

        org_images, labels = iter(dataloader).next()
        cam.batch_size = len(org_images)
        images = self.transform["val"](org_images)
        images = images.to(self.device)
        logits = self.forward(images).squeeze(1)
        pred = logits.sigmoid().detach().cpu().numpy() * 100
        labels = labels.cpu().numpy()

        grayscale_cam = cam(
            input_tensor=images, target_category=target_category, eigen_smooth=True
        )
        org_images = org_images.detach().cpu().numpy().transpose(0, 2, 3, 1) / 255.0
        return org_images, grayscale_cam, pred, labels

    def configure_optimizers(self):
        optimizer = eval(self.cfg.optimizer.name)(
            self.parameters(), **self.cfg.optimizer.params
        )
        scheduler = eval(self.cfg.scheduler.name)(
            optimizer, **self.cfg.scheduler.params
        )
        return [optimizer], [scheduler]

    def optimizer_zero_grad(self, epoch, batch_idx, optimizer, optimizer_idx):
        # https://pytorch-lightning.readthedocs.io/en/latest/guides/speed.html#set-grads-to-none
        optimizer.zero_grad(set_to_none=False)



In [ ]:
configs = [
    paw_config,
    bin_swin_config,
    center_crop_swin_config,
    bin_paw_config, 
    age_config, 
    breed_config, 
    adoption_speed_config, 
    gender_config,
    maturity_size_config
]

In [ ]:
paw_preds = []
img_embs = []
bin_swin_paw_preds = []
center_crop_swin_paw_preds = []
bin_paw_preds = []
bin_paw_preds_vecs = []
age_preds = []
breed_preds = []
adoption_speed_preds = []
adoption_speed_preds_vecs = []
gender_preds = []
maturity_size_preds = []
maturity_size_preds_vecs = []
for fold in range(max([config.n_splits for config in configs])):
    print("*"*20 + f"fold: {fold}" + "*"*20)
    
    # paw
    if fold in paw_config.train_folds:
        print("*"*20 + f"inference paw" + "*"*20)
        datamodule = PetfinderInferenceDataModule(test_df, paw_config)
        model = Model(paw_config)
        model.load_state_dict(
            torch.load(f"../input/petfinder-pawpularity-score-exp-{paw_config.exp_num}/fold_{fold}_best_loss.ckpt")["state_dict"] # FIX ME: pth か ckptか統一する
        )
        model = model.cuda().eval()

        trainer = pl.Trainer(
            logger=False,
            **paw_config.trainer,
        )
        output = trainer.predict(
            model=model,
            dataloaders=datamodule.predict_dataloader(),
            return_predictions=True,
        )
        oof_pred = []
        oof_img_emb = []
        for _output in output:
            oof_pred.append(_output[0])
            oof_img_emb.append(_output[1])
        oof_pred = torch.cat(oof_pred).numpy()
        oof_img_emb = torch.cat(oof_img_emb).numpy()
        paw_preds.append(oof_pred)
        img_embs.append(oof_img_emb)
        
        del datamodule, trainer, model, oof_pred, oof_img_emb, output
        gc.collect()
        torch.cuda.empty_cache()
        pl.utilities.memory.garbage_collection_cuda()

    # bin swin paw
    if fold in bin_swin_config.train_folds:
        print("*"*20 + f"inference bin swin paw" + "*"*20)
        datamodule = PetfinderInferenceDataModule(test_df, bin_swin_config)
        model = Model(bin_swin_config)
        model.load_state_dict(
            torch.load(f"../input/petfinder-pawpularity-score-exp-{bin_swin_config.exp_num}/fold_{fold}_best_loss.ckpt")["state_dict"]
        )
        model = model.cuda().eval()

        trainer = pl.Trainer(
            logger=False,
            **bin_swin_config.trainer,
        )
        output = trainer.predict(
            model=model,
            dataloaders=datamodule.predict_dataloader(),
            return_predictions=True,
        )
        oof_pred = []
        for _output in output:
            oof_pred.append(_output[0][0])
        oof_pred = torch.cat(oof_pred).numpy()
        bin_swin_paw_preds.append(oof_pred)
        
        del datamodule, trainer, model, oof_pred, output
        gc.collect()
        torch.cuda.empty_cache()
        pl.utilities.memory.garbage_collection_cuda()
        
    # center_crop_swin paw
    if fold in center_crop_swin_config.train_folds:
        print("*"*20 + f"inference center_crop_swin paw" + "*"*20)
        datamodule = PetfinderCenterInferenceDataModule(test_df, center_crop_swin_config)
        model = Model(center_crop_swin_config)
        model.load_state_dict(
            torch.load(f"../input/petfinder-pawpularity-score-exp-{center_crop_swin_config.exp_num}/fold_{fold}_best_loss.ckpt")["state_dict"]
        )
        model = model.cuda().eval()

        trainer = pl.Trainer(
            logger=False,
            **center_crop_swin_config.trainer,
        )
        output = trainer.predict(
            model=model,
            dataloaders=datamodule.predict_dataloader(),
            return_predictions=True,
        )
        oof_pred = []
        for _output in output:
            oof_pred.append(_output[0])
        oof_pred = torch.cat(oof_pred).numpy()
        center_crop_swin_paw_preds.append(oof_pred)
        
        del datamodule, trainer, model, oof_pred, output
        gc.collect()
        torch.cuda.empty_cache()
        pl.utilities.memory.garbage_collection_cuda()
       
    # bin_paw
    if fold in bin_paw_config.train_folds:
        print("*"*20 + f"inference bin_paw" + "*"*20)

        datamodule = PetfinderBinPawpularityInferenceDataModule(test_df, bin_paw_config)
        model = Model(bin_paw_config)
        model.load_state_dict(
            torch.load(f"../input/petfinder-pawpularity-score-exp-{bin_paw_config.exp_num}/fold_{fold}_best_loss.ckpt")["state_dict"]
        )
        model = model.cuda().eval()

        trainer = pl.Trainer(
            logger=False,
            **bin_paw_config.trainer,
        )

        output = trainer.predict(
            model=model,
            dataloaders=datamodule.predict_dataloader(),
            return_predictions=True,
        )
        oof_pred = []
        oof_pred_vec = []
        for _output in output:
            oof_pred.append(_output[0][0])
            oof_pred_vec.append(_output[0][1])
        oof_pred = torch.cat(oof_pred).numpy()
        bin_paw_preds.append(oof_pred)
        oof_pred_vec = torch.cat(oof_pred_vec).numpy()
        bin_paw_preds_vecs.append(oof_pred_vec)
        
        del datamodule, trainer, model, oof_pred, oof_pred_vec, output
        gc.collect()
        torch.cuda.empty_cache()
        pl.utilities.memory.garbage_collection_cuda()
        
    # age
    if fold in age_config.train_folds:
        print("*"*20 + f"inference age" + "*"*20)
        datamodule = PetfinderAgeInferenceDataModule(test_df, age_config)
        model = Model(age_config)
        model.load_state_dict(
            torch.load(f"../input/petfinder-pawpularity-score-exp-{age_config.exp_num}/fold_{fold}_best_loss.pth")
        )
        model = model.cuda().eval()

        trainer = pl.Trainer(
            logger=False,
            **age_config.trainer,
        )
        output = trainer.predict(
            model=model,
            dataloaders=datamodule.predict_dataloader(),
            return_predictions=True,
        )
        oof_pred = []
        for _output in output:
            oof_pred.append(_output[0])
        oof_pred = torch.cat(oof_pred)
        age_preds.append(oof_pred.numpy())
        
        del datamodule, trainer, model, oof_pred, output
        gc.collect()
        torch.cuda.empty_cache()
        pl.utilities.memory.garbage_collection_cuda()

    # breed
    if fold in breed_config.train_folds:
        print("*"*20 + f"inference breed" + "*"*20)

        datamodule = PetfinderBreedInferenceDataModule(test_df, breed_config)
        model = ClassificationModel(breed_config)
        model.load_state_dict(
            torch.load(f"../input/petfinder-pawpularity-score-exp-{breed_config.exp_num}/fold_{fold}_best_loss.pth")
        )
        model = model.cuda().eval()

        trainer = pl.Trainer(
            logger=False,
            **breed_config.trainer,
        )

        oof_pred = trainer.predict(
            model=model,
            dataloaders=datamodule.predict_dataloader(),
            return_predictions=True,
        )
        breed_preds.append(torch.cat(oof_pred).numpy())
        
        del datamodule, trainer, model, oof_pred
        gc.collect()
        torch.cuda.empty_cache()
        pl.utilities.memory.garbage_collection_cuda()

    # adoption_speed
    if fold in adoption_speed_config.train_folds:
        print("*"*20 + f"inference adoption_speed" + "*"*20)

        datamodule = PetfinderAdoptionSpeedInferenceDataModule(test_df, adoption_speed_config)
        model = Model(adoption_speed_config)
        model.load_state_dict(
            torch.load(f"../input/petfinder-pawpularity-score-exp-{adoption_speed_config.exp_num}/fold_{fold}_best_loss.ckpt")["state_dict"] # FIX ME: pth か ckptか統一する
        )
        model = model.cuda().eval()

        trainer = pl.Trainer(
            logger=False,
            **adoption_speed_config.trainer,
        )

        output = trainer.predict(
            model=model,
            dataloaders=datamodule.predict_dataloader(),
            return_predictions=True,
        )
        oof_pred = []
        oof_pred_vec = []
        for _output in output:
            oof_pred.append(_output[0][0])
            oof_pred_vec.append(_output[0][1])
        oof_pred = torch.cat(oof_pred).numpy()
        adoption_speed_preds.append(oof_pred)
        oof_pred_vec = torch.cat(oof_pred_vec).numpy()
        adoption_speed_preds_vecs.append(oof_pred_vec)
        
        del datamodule, trainer, model, oof_pred, oof_pred_vec, output
        gc.collect()
        torch.cuda.empty_cache()
        pl.utilities.memory.garbage_collection_cuda()

    # gender
    if fold in gender_config.train_folds:
        print("*"*20 + f"inference gender" + "*"*20)

        datamodule = PetfinderGenderInferenceDataModule(test_df, gender_config)
        model = ClassificationModel(gender_config)
        model.load_state_dict(
            torch.load(f"../input/petfinder-pawpularity-score-exp-{gender_config.exp_num}/fold_{fold}_best_loss.pth")
        )
        model = model.cuda().eval()

        trainer = pl.Trainer(
            logger=False,
            **gender_config.trainer,
        )

        oof_pred = trainer.predict(
            model=model,
            dataloaders=datamodule.predict_dataloader(),
            return_predictions=True,
        )
        gender_preds.append(torch.cat(oof_pred).numpy())
        
        del datamodule, trainer, model, oof_pred
        gc.collect()
        torch.cuda.empty_cache()
        pl.utilities.memory.garbage_collection_cuda()


    # Maturity Speed
    if fold in maturity_size_config.train_folds:
        print("*"*20 + f"inference MaturitySpeed" + "*"*20)

        datamodule = PetfinderMaturitySizeInferenceDataModule(test_df, maturity_size_config)
        model = Model(maturity_size_config)
        model.load_state_dict(
            torch.load(f"../input/petfinder-pawpularity-score-exp-{maturity_size_config.exp_num}/fold_{fold}_best_loss.ckpt")["state_dict"] 
        )
        model = model.cuda().eval()

        trainer = pl.Trainer(
            logger=False,
            **adoption_speed_config.trainer,
        )

        output = trainer.predict(
            model=model,
            dataloaders=datamodule.predict_dataloader(),
            return_predictions=True,
        )
        oof_pred = []
        oof_pred_vec = []
        for _output in output:
            oof_pred.append(_output[0][0])
            oof_pred_vec.append(_output[0][1])
        oof_pred = torch.cat(oof_pred).numpy()
        maturity_size_preds.append(oof_pred)
        oof_pred_vec = torch.cat(oof_pred_vec).numpy()
        maturity_size_preds_vecs.append(oof_pred_vec)
        
        del datamodule, trainer, model, oof_pred, oof_pred_vec, output
        gc.collect()
        torch.cuda.empty_cache()
        pl.utilities.memory.garbage_collection_cuda()


In [ ]:
test_df["oof_pred"] = np.mean(paw_preds, axis=0)

emb_cols = [ f"emb_{i}" for i in range(len(img_embs[0][0]))]
test_df.loc[:, emb_cols] = np.mean(img_embs, axis=0)

test_df["bin_swin_oof_pred"] = np.mean(bin_swin_paw_preds, axis=0)

test_df["center_crop_swin_oof_pred"] = np.mean(center_crop_swin_paw_preds, axis=0)


test_df["bin_paw_oof_pred"] = np.mean(bin_paw_preds, axis=0)
bin_paw_pred_vec_cols = [f"bin_paw_pred_vec_{i}" for i in range(bin_paw_preds_vecs[0].shape[1])]
test_df.loc[:, bin_paw_pred_vec_cols] = np.mean(bin_paw_preds_vecs, axis=0)
test_df["bin_paw_oof_pred_bin"] = pd.cut(
                test_df["bin_paw_oof_pred"],
                3,
                labels=False,
            )

test_df["Age"] = np.mean(age_preds, axis=0)
test_df["Age_bin"] = pd.cut(
    test_df["Age"], [-np.inf, 7, 13, 48, np.inf], labels=False
)
test_df["Age_year"] = test_df["Age"] // 12

breed_cols = [f"breed_pred_{i}" for i in range(len(breed_preds[0][0]))]
test_df.loc[:, breed_cols] = np.mean(breed_preds, axis=0)
test_df["breed"] = np.argmax(np.mean(breed_preds, axis=0), axis=1)

pred_vec_cols = [f"adoption_speed_pred_vec_{i}" for i in range(4)]
test_df["AdoptionSpeed"] = np.mean(adoption_speed_preds, axis=0)
test_df.loc[:, pred_vec_cols] = np.mean(adoption_speed_preds_vecs, axis=0)

gender_cols = [f"gender_pred_{i}" for i in range(len(gender_preds[0][0]))]
test_df.loc[:, gender_cols] = np.mean(gender_preds, axis=0)
test_df["gender"] = np.argmax(np.mean(gender_preds, axis=0), axis=1)

pred_vec_cols = [f"maturity_size_pred_vec_{i}" for i in range(3)]
test_df["MaturitySize"] = np.mean(maturity_size_preds, axis=0)
test_df.loc[:, pred_vec_cols] = np.mean(maturity_size_preds_vecs, axis=0)
test_df["MaturitySize_bin"] = pd.cut(
    test_df["MaturitySize"], 4, labels=False
)

In [ ]:
test_df.head().T

In [ ]:
bbox_info = pd.read_csv("/kaggle/working/bbox_info.csv")[
                [
                    "Id",
                    "height",
                    "width",
                    "aspect_ratio",
                    "area",
                    "num_dog",
                    "num_cat",
                    "num_teddy_bear",
                    "num_person",
                    "num_dog_cat",
                    "num_dog_cat_teddy_bear",
                ]
            ]
bbox_info

In [ ]:
test_df[[col for col in test_df.columns if "Id" in col]]

In [ ]:
test_df = pd.merge(test_df, bbox_info, how="left", left_on="org_Id", right_on="Id")
del bbox_info
gc.collect()

In [ ]:
test_df = test_df[[col for col in test_df.columns if col != "Id_y"]].rename(columns={"Id_x": "Id"})

In [ ]:
test_df[[col for col in test_df.columns if "Id" in col]]

In [ ]:
"""types."""
from typing import Union

import numpy as np
import pandas as pd


try:
    from cudf import DataFrame as CDataFrame
    from cudf import Series as CSeries
    from cupy import ndarray as CNDArray
except ImportError:
    CDataFrame = None
    CSeries = None
    CNDArray = None

XDataFrame = Union[CDataFrame, pd.DataFrame]
XSeries = Union[CSeries, np.ndarray]
XSeriesB = Union[CSeries, pd.Series]
XNDArray = Union[CNDArray, np.ndarray]

import importlib
from typing import Any, List, Dict

class TransformerMixin:
    """Mixin class for `xfeat.encoder`."""

    def fit(self, input_df: XDataFrame) -> None:
        """Fit to data frame.

        Args:
            input_df (XDataFrame): Input data frame.
        """
        raise NotImplementedError("Not implemented yet.")

    def transform(self, input_df: XDataFrame) -> XDataFrame:
        """Transform data frame.

        Args:
            input_df (XDataFrame): Input data frame.
        Returns:
            XDataFrame : Output data frame.
        """
        raise NotImplementedError("Not implemented yet.")

    def fit_transform(self, input_df: XDataFrame) -> XDataFrame:
        """Fit to data frame, then transform it.

        Args:
            input_df (XDataFrame): Input data frame.
        Returns:
            XDataFrame : Output data frame.
        """
        self.fit(input_df)
        return self.transform(input_df)


from itertools import combinations
from typing import List, Optional


class ConcatCombination(TransformerMixin):
    """Generate combination of string columns.

    Example:
        ::

            >>> import pandas as pd
            >>> from xfeat import ConcatCombination
            >>> df = pd.DataFrame({
              "col1": ["a", "b"],
              "col2": ["@", "%"],
              "col3": ["X", "Y"]
            })
            >>> encoder = ConcatCombination()
            >>> encoder.fit_transform(df)
              col1 col2 col3 col1col2_combi col1col3_combi col2col3_combi
            0    a    @    X             a@             aX             @X
            1    b    %    Y             b%             bY             %Y

            >>> encoder = ConcatCombination(output_suffix="", drop_origin=True)
            >>> encoder.fit_transform(df)
              col1col2 col1col3 col2col3
            0       a@       aX       @X
            1       b%       bY       %Y

            >>> encoder = ConcatCombination(output_suffix="", drop_origin=True, r=3)
            >>> encoder.fit_transform(df)
              col1col2col3
            0          a@X
            1          b%Y

    Args:
        input_cols (Optional[List[str]]):
            Input column names. The default uses all columns of the input data frame.

        include_cols (Optional[List[str]]):
            Columns of the input data frame that are passed on to the output data frame.
            Defaults: None.

        output_prefix (str):
            Prefix of output column name. Defaults: `""`.

        output_suffix (str):
            Suffix of output column name. Defaults: `"_combi"`.

        drop_origin (bool):
            Drop the original column names. Defaults: `False`.

        fillna (str):
            To concatenate the string columns, the missing values are replaced with the
            string value `fillna`. Defaults: `"_NaN_"`.

        r (int):
            Length of combinations. Default: `2`.
    """

    def __init__(
        self,
        input_cols: Optional[List[str]] = None,
        include_cols: Optional[List[str]] = None,
        output_prefix: str = "",
        output_suffix: str = "_combi",
        drop_origin: bool = False,
        fillna: str = "_NaN_",
        r: int = 2,
    ):
        self._input_cols = input_cols or []
        self._include_cols = include_cols or []
        self._output_prefix = output_prefix
        self._output_suffix = output_suffix
        self._r = r
        self._fillna = fillna
        self._drop_origin = drop_origin

    def fit_transform(self, input_df: XDataFrame) -> XDataFrame:
        """Fit to data frame, then transform it.

        Args:
            input_df (XDataFrame): Input data frame.

        Returns:
            XDataFrame : Output data frame.
        """
        input_cols = self._input_cols

        if not input_cols:
            self._input_cols = [
                col
                for col in input_df.columns.tolist()
                if col not in self._include_cols
            ]

        return self.transform(input_df)

    def transform(self, input_df: XDataFrame) -> XDataFrame:
        """Transform data frame.

        Args:
            input_df (XDataFrame): Input data frame.

        Returns:
            XDataFrame : Output data frame.
        """
        cols = []

        n_fixed_cols = len(self._include_cols)
        df = input_df.copy()

        for cols_pairs in combinations(self._input_cols, r=self._r - n_fixed_cols):
            fixed_cols_str = "".join(self._include_cols)
            pairs_cols_str = "".join(cols_pairs)
            new_col = (
                self._output_prefix
                + fixed_cols_str
                + pairs_cols_str
                + self._output_suffix
            )
            cols.append(new_col)

            concat_cols = self._include_cols + list(cols_pairs)
            new_ser = None
            for col in concat_cols:
                if new_ser is None:
                    new_ser = df[col].fillna(self._fillna).copy()
                else:
                    new_ser = new_ser + df[col].fillna(self._fillna)

            df[new_col] = new_ser

        if self._drop_origin:
            return df[cols]

        return df


In [ ]:
from sklearn.preprocessing import LabelEncoder

groupby_keys = [
    "MaturitySize_bin",
    "Age_bin",
    "breed",
    "gender",
]
new_cat_df = pd.concat(
    [
        ConcatCombination(drop_origin=True, r=r).fit_transform(
            test_df[groupby_keys].astype(str).fillna("none")
        )
        for r in [
            2,
            3,
            4,
        ]
    ],
    axis="columns",
)

for col in new_cat_df.columns:
    le = LabelEncoder()
    new_cat_df[col] = le.fit_transform(new_cat_df[col])

In [ ]:
new_cat_df

In [ ]:
test_df = pd.concat(
    [test_df, new_cat_df],
    axis="columns",
)
combi_cat_cols = new_cat_df.columns.to_list()
del new_cat_df
gc.collect()

In [ ]:
groupby_dict = []
num_var_list = [
    "Age",
    "MaturitySize",
    "AdoptionSpeed",
    "adoption_speed_pred_vec_0",
    "adoption_speed_pred_vec_1",
    "adoption_speed_pred_vec_2",
    "adoption_speed_pred_vec_3",
    "gender_pred_0",
    "gender_pred_1",
    "gender_pred_2",
] + [col for col in test_df.keys() if "breed_pred_" in col]
num_stats_list = [
    "mean",
    "var",
    "std",
    "min",
    "max",
    "sum",
]
for key in groupby_keys + combi_cat_cols + ["bin_paw_oof_pred_bin"]:
    groupby_dict.append(
        {
            "key": [key],
            "var": num_var_list,
            "agg": num_stats_list,
        }
    )

In [ ]:
from tqdm import tqdm_notebook as tqdm

In [ ]:
class GroupbyTransformer:
    def __init__(self, param_dict=None):
        self.param_dict = param_dict

    def _get_params(self, p_dict):
        key = p_dict["key"]
        if "var" in p_dict.keys():
            var = p_dict["var"]
        else:
            var = self.var
        if "agg" in p_dict.keys():
            agg = p_dict["agg"]
        else:
            agg = self.agg
        if "on" in p_dict.keys():
            on = p_dict["on"]
        else:
            on = key
        return key, var, agg, on

    def _aggregate(self, dataframe):
        self.features = []
        for param_dict in tqdm(self.param_dict):
            key, var, agg, on = self._get_params(param_dict)
            all_features = list(set(key + var))
            new_features = self._get_feature_names(key, var, agg)
            features = (
                dataframe[all_features].groupby(key)[var].agg(agg).reset_index()
            )
            features.columns = key + new_features
            self.features.append(features)
        return self

    def _merge(self, dataframe, merge=True):
        for param_dict, features in tqdm(
            zip(self.param_dict, self.features), total=len(self.features)
        ):
            key, var, agg, on = self._get_params(param_dict)
            if merge:
                dataframe = dataframe.merge(features, how="left", on=on)
            else:
                new_features = self._get_feature_names(key, var, agg)
                dataframe = pd.concat([dataframe, features[new_features]], axis=1)
        return dataframe

    def transform(self, dataframe):
        self._aggregate(dataframe)
        return self._merge(dataframe, merge=True)

    def _get_feature_names(self, key, var, agg):
        _agg = []
        for a in agg:
            if not isinstance(a, str):
                _agg.append(a.__name__)
            else:
                _agg.append(a)
        return ["_".join([a, v, "groupby"] + key) for v in var for a in _agg]

    def get_feature_names(self):
        self.feature_names = []
        for param_dict in self.param_dict:
            key, var, agg, on = self._get_params(param_dict)
            self.feature_names += self._get_feature_names(key, var, agg)
        return self.feature_names

    def get_numerical_features(self):
        return self.get_feature_names()


class DiffGroupbyTransformer(GroupbyTransformer):
    def _aggregate(self):
        raise NotImplementedError

    def _merge(self):
        raise NotImplementedError

    def transform(self, dataframe):
        for param_dict in tqdm(self.param_dict):
            key, var, agg, on = self._get_params(param_dict)
            for a in agg:
                for v in var:
                    if not isinstance(a, str):
                        new_feature = "_".join(["diff", a.__name__, v, "groupby"] + key)
                        base_feature = "_".join([a.__name__, v, "groupby"] + key)
                    else:
                        new_feature = "_".join(["diff", a, v, "groupby"] + key)
                        base_feature = "_".join([a, v, "groupby"] + key)
#                     print(new_feature)
                    if str(dataframe[v].dtype) == "category":
                        dataframe[new_feature] = dataframe[base_feature] - dataframe[
                            v
                        ].astype(int)
                    else:
                        dataframe[new_feature] = dataframe[base_feature] - dataframe[v]

        return dataframe

    def _get_feature_names(self, key, var, agg):
        _agg = []
        for a in agg:
            if not isinstance(a, str):
                _agg.append(a.__name__)
            else:
                _agg.append(a)
        return ["_".join(["diff", a, v, "groupby"] + key) for v in var for a in _agg]


class RatioGroupbyTransformer(GroupbyTransformer):
    def _aggregate(self):
        raise NotImplementedError

    def _merge(self):
        raise NotImplementedError

    def transform(self, dataframe):
        for param_dict in tqdm(self.param_dict):
            key, var, agg, on = self._get_params(param_dict)
            for a in agg:
                for v in var:
                    if not isinstance(a, str):
                        new_feature = "_".join(
                            ["ratio", a.__name__, v, "groupby"] + key
                        )
                        base_feature = "_".join([a.__name__, v, "groupby"] + key)
                    else:
                        new_feature = "_".join(["ratio", a, v, "groupby"] + key)
                        base_feature = "_".join([a, v, "groupby"] + key)
#                     print(new_feature)

                    if str(dataframe[v].dtype) == "category":
                        dataframe[new_feature] = dataframe[base_feature] / dataframe[
                            v
                        ].astype(int)
                    else:
                        dataframe[new_feature] = dataframe[base_feature] / dataframe[v]
        return dataframe

    def _get_feature_names(self, key, var, agg):
        _agg = []
        for a in agg:
            if not isinstance(a, str):
                _agg.append(a.__name__)
            else:
                _agg.append(a)
        return ["_".join(["ratio", a, v, "groupby"] + key) for v in var for a in _agg]


In [ ]:
groupby = GroupbyTransformer(groupby_dict)
test_df = groupby.transform(test_df)

groupby = DiffGroupbyTransformer(groupby_dict)
test_df = groupby.transform(test_df)

groupby = RatioGroupbyTransformer(groupby_dict)
test_df = groupby.transform(test_df)

In [ ]:
print("end")

In [ ]:
oof_train = pd.read_csv(f"../input/petfinder-pawpularity-score-xgb-exp-{GBDT_EXP_NUM}/oof_pred.csv")
oof_train.head()

In [ ]:
gbdt_config.categorical_cols

In [ ]:
y_train = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")["Pawpularity"].values
y_train

In [ ]:
if gbdt_config["pre_process"]["xentropy"]:
    y_train = y_train / 100
elif gbdt_config["pre_process"]["tweedie"]:
    y_train = 100 - y_train

In [ ]:
y_train

In [ ]:
cols = load_pickle(f"../input/petfinder-pawpularity-score-xgb-exp-{GBDT_EXP_NUM}/cols.pkl")

In [ ]:
# for col in cols:
#     print(col)

In [ ]:
models = load_pickle(f"../input/petfinder-pawpularity-score-xgb-exp-{GBDT_EXP_NUM}/model.pkl")

In [ ]:
models

In [ ]:
from cuml.preprocessing.TargetEncoder import TargetEncoder

if gbdt_config["target_encoding"]:
    cat_cols = gbdt_config["categorical_cols"]
    for cat_col in tqdm(cat_cols):
        encoder = TargetEncoder(n_folds=4, smooth=0.3)
        encoder.fit(oof_train[cat_col], y_train)
        test_df[cat_col + "_TE"] = encoder.transform(test_df[cat_col])

In [ ]:
set(cols) - set(test_df.columns)

In [ ]:
set(test_df.columns) - set(cols)

In [ ]:
import xgboost as xgb
xgb.__version__

In [ ]:
test_preds = np.zeros(len(test_df))
for fold in range(len(models)):
    model = models[fold]

    test_preds += (
        model.predict(test_df[cols].values, ntree_limit=model.best_ntree_limit) / len(models)
    )

In [ ]:
test_df["xgb_oof_pred_xentropy_065"] = test_preds * 100

In [ ]:
GBDT_EXP_NUM = "054"

In [ ]:
gbdt_config = OmegaConf.load(
    f"../input/petfinder-pawpularity-score-xgb-exp-{GBDT_EXP_NUM}/output.json"
)
# pprint(gbdt_config)

In [ ]:
oof_train = pd.read_csv(f"../input/petfinder-pawpularity-score-xgb-exp-{GBDT_EXP_NUM}/oof_pred.csv")
oof_train.head()

In [ ]:
gbdt_config.categorical_cols

In [ ]:
y_train = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")["Pawpularity"].values
y_train

In [ ]:
if gbdt_config ["pre_process"]["xentropy"]:
    y_train = y_train / 100
elif gbdt_config["pre_process"]["tweedie"]:
    y_train = 100 - y_train

In [ ]:
y_train

In [ ]:
cols = load_pickle(f"../input/petfinder-pawpularity-score-xgb-exp-{GBDT_EXP_NUM}/cols.pkl")

In [ ]:
# for col in cols:
#     print(col)

In [ ]:
models = load_pickle(f"../input/petfinder-pawpularity-score-xgb-exp-{GBDT_EXP_NUM}/model.pkl")

In [ ]:
from cuml.preprocessing.TargetEncoder import TargetEncoder

if gbdt_config["target_encoding"]:
    cat_cols = gbdt_config["categorical_cols"]
    for cat_col in tqdm(cat_cols):
        encoder = TargetEncoder(n_folds=4, smooth=0.3)
        encoder.fit(oof_train[cat_col], y_train)
        test_df[cat_col + "_TE"] = encoder.transform(test_df[cat_col])

In [ ]:
set(cols) - set(test_df.columns)

In [ ]:
set(test_df.columns) - set(cols)

In [ ]:
test_preds = np.zeros(len(test_df))
for fold in range(len(models)):
    model = models[fold]

    test_preds += (
        model.predict(test_df[cols].values, ntree_limit=model.best_ntree_limit) / len(models)
    )

In [ ]:
test_df["xgb_oof_pred_xentropy_054"] = test_preds * 100

In [ ]:
GBDT_EXP_NUM = "064"

In [ ]:
gbdt_config = OmegaConf.load(
    f"../input/petfinder-pawpularity-score-xgb-exp-{GBDT_EXP_NUM}/output.json"
)
# pprint(gbdt_config)

In [ ]:
oof_train = pd.read_csv(f"../input/petfinder-pawpularity-score-xgb-exp-{GBDT_EXP_NUM}/oof_pred.csv")
oof_train.head()

In [ ]:
gbdt_config.categorical_cols

In [ ]:
y_train = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")["Pawpularity"].values
y_train

In [ ]:
gbdt_config["target"]

In [ ]:
gbdt_config["pre_process"]["xentropy"]

In [ ]:
gbdt_config["pre_process"]["tweedie"]

In [ ]:
if gbdt_config["target"] == "residual":
    y_train = y_train - (oof_train["ensemble_pred"] - oof_train["xgb_oof_pred"])
    
if gbdt_config["pre_process"]["xentropy"]:
    y_train = y_train / 100
elif gbdt_config["pre_process"]["tweedie"]:
    y_train = 100 - y_train

In [ ]:
y_train

In [ ]:
cols = load_pickle(f"../input/petfinder-pawpularity-score-xgb-exp-{GBDT_EXP_NUM}/cols.pkl")

In [ ]:
# for col in cols:
#     print(col)

In [ ]:
models = load_pickle(f"../input/petfinder-pawpularity-score-xgb-exp-{GBDT_EXP_NUM}/model.pkl")

In [ ]:
from cuml.preprocessing.TargetEncoder import TargetEncoder

if gbdt_config["target_encoding"]:
    cat_cols = gbdt_config["categorical_cols"]
    for cat_col in tqdm(cat_cols):
        encoder = TargetEncoder(n_folds=4, smooth=0.3)
        encoder.fit(oof_train[cat_col], y_train)
        test_df[cat_col + "_TE"] = encoder.transform(test_df[cat_col])

In [ ]:
set(cols) - set(test_df.columns)

In [ ]:
set(test_df.columns) - set(cols)

In [ ]:
test_preds = np.zeros(len(test_df))
for fold in range(len(models)):
    model = models[fold]

    test_preds += (
        model.predict(test_df[cols].values, ntree_limit=model.best_ntree_limit) / len(models)
    )

In [ ]:
test_df[[col for col in test_df.columns if "oof" in col]]

In [ ]:
pd.Series(test_preds).describe()

In [ ]:
test_df["xgb_oof_pred_residual_064"] = test_preds + test_df["oof_pred"]

In [ ]:
test_df[[col for col in test_df.columns if ("oof" in col) & ("groupby" not in col) & ("TE" not in col)]]

In [ ]:
# test_df[["oof_pred", "xgb_oof_pred_xentropy", "tweedie_xgb_oof_pred_xentropy"]]
test_df[["oof_pred", "bin_swin_oof_pred", "center_crop_swin_oof_pred", "xgb_oof_pred_xentropy_054", "xgb_oof_pred_residual_064", "xgb_oof_pred_xentropy_065"]]

In [ ]:
weights = np.array([-3.04675469,  1.37952536,  2.31896654,  0.26659276,  3.77603365,1.27495351])

In [ ]:
test_df["Pawpularity"] = np.dot(test_df[["oof_pred", "bin_swin_oof_pred", "center_crop_swin_oof_pred", "xgb_oof_pred_xentropy_054", "xgb_oof_pred_residual_064", "xgb_oof_pred_xentropy_065"]].values / 6, weights)

In [ ]:
# test_df["Pawpularity"] = 0.5*test_df["oof_pred"] + 0.5*test_df["xgb_oof_pred_xentropy"]
# test_df["Pawpularity"] = test_df["oof_pred"] 

In [ ]:
karunru = test_df[["org_Id", "Pawpularity"]].rename(columns={"org_Id": "Id"})

# クーラビ part

In [ ]:
import sys
sys.path.append('../input/timm-3monthsold/pytorch-image-models-master 2')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm
import os, gc
import random
from PIL import Image
import tifffile as tiff
import cv2
import zipfile
import collections
from PIL import Image
from sklearn import preprocessing
from random import randint
from glob import glob
import shutil
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torch.nn.modules.loss import _WeightedLoss
import torchvision
from torchvision import transforms
import albumentations as A
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.optim.lr_scheduler import CosineAnnealingLR
import warnings
warnings.filterwarnings("ignore")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed = 2020
seed_everything(seed)
sz1 = 224
sz2 = 384
NFOLDS = 5
pet_num = 4

#ImageNet
mean = np.array([[[0.485, 0.456, 0.406]]])
std = np.array([[[0.229, 0.224, 0.225]]])

In [ ]:
test_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')
#test_df = pd.concat([pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')]*900, ignore_index=True)
test_df['class'] = -1
test_df['conf'] = -1
test_ids = test_df.Id.to_list()
#test_ids = ['0013fd999caf9a3efe1352ca1b0d937e', '0009c66b9439883ba2750fb825e1d7db', '0007de18844b0dbbb5e1f607da0606e0']
test_dir = "/kaggle/input/petfinder-pawpularity-score/test/"
#test_dir = '/kaggle/input/petfinder2-sample-images/'
shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5')

In [ ]:
tmp = pd.DataFrame(columns=test_df.columns)
tmp

In [ ]:
!python detect.py\
--weights /kaggle/input/ultralyticsyolov5aweights/yolov5x.pt\
--class 15 16\
--img 512\
--conf 0.3\
--iou 0.5\
--source $test_dir\
--name inference\
--save-txt --save-conf --exist-ok

In [ ]:
os.chdir('/kaggle/working')
save_dir = f'/kaggle/working/crop_images/'
os.makedirs(save_dir, exist_ok=True)

for n, image_id in tqdm(enumerate(test_ids)):
    orig_image = cv2.imread(f'/kaggle/input/petfinder-pawpularity-score/test/{image_id}.jpg')
    orig_image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
    height = orig_image.shape[0]
    width = orig_image.shape[1]
    try:
        file_path = f'/kaggle/working/yolov5/runs/detect/inference/labels/{image_id}.txt'
        f = open(file_path, 'r')
        data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]
        data = data[np.argsort(data[:, 1])[::-1]]  #sort by conf
        for i, d in enumerate(data):
            xmin = int((d[2]-d[4]/2)*width)
            ymin = int((d[3]-d[5]/2)*height)
            xmax = int((d[2]+d[4]/2)*width)
            ymax = int((d[3]+d[5]/2)*height)
            width_half = (xmax - xmin) // 2
            height_half = (ymax - ymin) // 2
            r = np.maximum(width_half, height_half)
            xc = (xmin + xmax) // 2
            yc = (ymin + ymax) // 2
            final_xmin = np.maximum(xc-r, 0)
            final_ymin = np.maximum(yc-r, 0)
            final_xmax = np.minimum(xc+r, width)
            final_ymax = np.minimum(yc+r, height)
            crop_img = orig_image[final_ymin:final_ymax, final_xmin:final_xmax, :]
            np.save(save_dir + f'{image_id}-{i}', crop_img.astype(np.uint8))
            df = pd.DataFrame(columns=test_df.columns)
            df.loc[0, 'Id'] = f'{image_id}-{i}'
            df.loc[0, 'class'] = d[0]
            df.loc[0, 'conf'] = d[1]
            tmp = tmp.append(df, ignore_index=True)
            
    except:
        np.save(save_dir + f'{image_id}-0', orig_image.astype(np.uint8))
        df = pd.DataFrame(columns=test_df.columns)
        df.loc[0, 'Id'] = f'{image_id}-0'
        df.loc[0, 'class'] = 'NA'
        df.loc[0, 'conf'] = 'NA'
        tmp = tmp.append(df, ignore_index=True)

In [ ]:
tmp = tmp.rename(columns={'Id': 'Id2'})
tmp['img_idx'] = tmp['Id2'].apply(lambda x: x.split('-')[1])
tmp['Id'] = tmp['Id2'].apply(lambda x: x.split('-')[0])
tmp

In [ ]:
class Dataset(Dataset):
    def __init__(self, df, size, transform=None):
        self.df = df
        self.size = size
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        image_id = self.df.loc[idx, 'Id2']
        img = np.load(f'/kaggle/working/crop_images/{image_id}.npy')
        img = cv2.resize(img, (self.size, self.size)).astype(np.float32)
        
        if self.transform:
            sample = self.transform(image=img)
            img = sample['image']
        
        img = (img/255.0 - mean) / std
        img = np.transpose(img, (2, 0, 1))
        img = torch.from_numpy(img)

        return img
    
    
class Dataset2(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        image_id = self.df.loc[idx, 'Id']
        img = np.load(f'/kaggle/working/crop_images/{image_id}.npy').astype(np.float32)
        
        if self.transform:
            sample = self.transform(image=img)
            img = sample['image']
        
        img = (img/255.0 - mean) / std
        img = np.transpose(img, (2, 0, 1))
        img = torch.from_numpy(img)

        return img
    

def inference_fn1(data_loader, model, device):
    model.eval()    
    val_preds = []
    
    for i, x in enumerate(data_loader):
        img = x
        img = img.to(device, dtype=torch.float)
        
        with torch.no_grad():
            pred = model(img)
            val_preds.append(nn.Softmax()(pred).detach().cpu().numpy())
            
    val_preds = np.concatenate(val_preds)
                
    return val_preds


def inference_fn2(data_loader, model, device):
    model.eval()    
    val_preds = []
    
    for i, x in enumerate(data_loader):
        img = x
        img = img.to(device, dtype=torch.float)
        
        with torch.no_grad():
            pred = model(img)
            val_preds.append(nn.Sigmoid()(pred).detach().cpu().numpy() * 100)
            
    val_preds = np.concatenate(val_preds)
                
    return val_preds


class Model(nn.Module):
    def __init__(self, model_name=None, pretrained=False, num_classes=100):
        super().__init__()
        self.model = timm.create_model(
            model_name=model_name, 
            in_chans=3, 
            pretrained=pretrained
            )
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features, num_classes)

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
#224x224
#5folds
#cropped imgs
test_ds = Dataset(df=tmp, size=sz1, transform=None)
test_dl = DataLoader(dataset=test_ds, batch_size=256, shuffle=False, num_workers=2)
target_cols = np.arange(1, 101)
predictions1 = 0
predictions2 = 0

for fold in range(NFOLDS):
    print(f'======== 224 yolo FOLD:{fold} inference ========')
    
    #SwinT type1: 100 outputs
    model1 = Model(model_name='swin_large_patch4_window7_224_in22k', num_classes=100)
    model1.load_state_dict(torch.load(f'/kaggle/input/petfinder2-swint1-weight-016/swint1_fold_{fold}.pth'))
    model1.to(device)
    prediction1 = inference_fn1(test_dl, model1, device)
    predictions1 += (target_cols.reshape(-1, 100) * prediction1.reshape(-1, 100)).sum(axis=1) / NFOLDS
    
    #SwinT type2: 1 output
    model2 = Model(model_name='swin_large_patch4_window7_224_in22k', num_classes=1)
    model2.model.head = nn.Sequential(
        nn.Linear(model2.model.head.in_features, 128),
        nn.Dropout(0.0),
        nn.Linear(128, 64),
        nn.Linear(64, 1)
    )
    model2.load_state_dict(torch.load(f'/kaggle/input/petfinder2-swint2-weight-011/swint2_fold_{fold}.pth'))
    model2.to(device)
    prediction2 = inference_fn2(test_dl, model2, device).flatten()
    predictions2 += prediction2 / NFOLDS
    
    del model1, model2, prediction1, prediction2
    gc.collect()
    
tmp['pred1'] = predictions1
tmp['pred2'] = predictions2

del test_ds, test_dl, predictions1, predictions2
gc.collect()

In [ ]:
#384x384
#5folds
#cropped imgs
test_ds = Dataset(df=tmp, size=sz2, transform=None)
test_dl = DataLoader(dataset=test_ds, batch_size=64, shuffle=False, num_workers=2)
target_cols = np.arange(1, 101)
predictions3 = 0
predictions4 = 0

for fold in range(NFOLDS):
    print(f'======== 384 yolo FOLD:{fold} inference ========')
    
    #SwinT type1: 100 outputs
    model1 = Model(model_name='swin_large_patch4_window12_384_in22k', num_classes=100)
    model1.load_state_dict(torch.load(f'/kaggle/input/petfinder2-swint1-weight-068/swint1_fold_{fold}.pth'))
    model1.to(device)
    prediction3 = inference_fn1(test_dl, model1, device)
    predictions3 += (target_cols.reshape(-1, 100) * prediction3.reshape(-1, 100)).sum(axis=1) / NFOLDS
    
    #SwinT type2: 1 output
    model2 = Model(model_name='swin_large_patch4_window12_384_in22k', num_classes=1)
    model2.model.head = nn.Sequential(
        nn.Linear(model2.model.head.in_features, 128),
        nn.Dropout(0.0),
        nn.Linear(128, 64),
        nn.Linear(64, 1)
    )
    model2.load_state_dict(torch.load(f'/kaggle/input/petfinder2-swint2-weight-028/swint2_fold_{fold}.pth'))
    model2.to(device)
    prediction4 = inference_fn2(test_dl, model2, device).flatten()
    predictions4 += prediction4 / NFOLDS
    
    del model1, model2, prediction3, prediction4
    gc.collect()
    
tmp['pred3'] = predictions3
tmp['pred4'] = predictions4

del test_ds, test_dl, predictions3, predictions4
gc.collect()

In [ ]:
tmp

In [ ]:
#postprocessing by multi-pets
test_df = tmp.groupby('Id').head(pet_num).groupby('Id').mean().reset_index()
test_df

In [ ]:
sub = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
coorabi = pd.concat(
    [sub, 
     test_df[['pred1']], 
     test_df[['pred2']], 
     test_df[['pred3']], 
     test_df[['pred4']]
    ], axis=1
)
coorabi

In [ ]:
!rm -r /kaggle/working/yolov5
!rm -r /kaggle/working/crop_images

In [ ]:
#fastai
#224x224
#5folds
#original imgs
import fastai
from fastai.vision.all import *
from fastai.callback.all import *
import torchvision.models as torch_models

def petfinder_rmse(input, target):
    return 100 * torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

test_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')
test_df['Path'] = '../input/petfinder-pawpularity-score/test/' + test_df['Id'] + '.jpg'
predictions5 = 0

for fold in range(NFOLDS):
    print(f'======== fastai FOLD:{fold} inference ========')
    learn = load_learner(fname = Path(f'/kaggle/input/petfinder2-swint3-weight-001/swint3_fold_{fold}.pkl'), cpu=False)
    test_dl = learn.dls.test_dl(test_df)
    preds, _ = learn.get_preds(dl=test_dl)
    predictions5 += preds * 100 / NFOLDS
    del learn, test_dl, preds
    gc.collect()
    
coorabi['pred5'] = predictions5

del predictions5
gc.collect()

In [ ]:
#224x224
#5folds
#center crop
#binary classifier
test_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')
test_ids = test_df.Id.to_list()
os.chdir('/kaggle/working')
save_dir = f'/kaggle/working/crop_images/'
os.makedirs(save_dir, exist_ok=True)

for n, image_id in tqdm(enumerate(test_ids)):
    orig_image = cv2.imread(f'/kaggle/input/petfinder-pawpularity-score/test/{image_id}.jpg')
    orig_image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
    height = orig_image.shape[0]
    width = orig_image.shape[1]
    xc = width // 2
    yc = height // 2
    r = np.minimum(xc, yc)
    xmin = np.maximum(xc-r, 0)
    ymin = np.maximum(yc-r, 0)
    xmax = np.minimum(xc+r, width)
    ymax = np.minimum(yc+r, height)
    crop_img = orig_image[ymin:ymax, xmin:xmax, :]
    crop_img = cv2.resize(crop_img, (sz1, sz1)).astype(np.uint8)
    np.save(save_dir + f'{image_id}', crop_img)

test_ds = Dataset2(df=coorabi, transform=None)
test_dl = DataLoader(dataset=test_ds, batch_size=256, shuffle=False, num_workers=2)
target_cols = np.arange(1, 101)
predictions6 = 0
predictions7 = 0
binary = 0

for fold in range(NFOLDS):
    print(f'======== 224 center FOLD:{fold} inference ========')
    
    #SwinT type1: 100 outputs
    model1 = Model(model_name='swin_large_patch4_window7_224_in22k', num_classes=100)
    model1.load_state_dict(torch.load(f'/kaggle/input/petfinder2-swint1-weight-076/swint1_fold_{fold}.pth'))
    model1.to(device)
    prediction6 = inference_fn1(test_dl, model1, device)
    predictions6 += (target_cols.reshape(-1, 100) * prediction6.reshape(-1, 100)).sum(axis=1) / NFOLDS
    
    #SwinT type2: 1 output
    model2 = Model(model_name='swin_large_patch4_window7_224_in22k', num_classes=1)
    model2.model.head = nn.Sequential(
        nn.Linear(model2.model.head.in_features, 128),
        nn.Dropout(0.0),
        nn.Linear(128, 64),
        nn.Linear(64, 1)
    )
    model2.load_state_dict(torch.load(f'/kaggle/input/petfinder2-swint2-weight-036/swint2_fold_{fold}.pth'))
    model2.to(device)
    prediction7 = inference_fn2(test_dl, model2, device).flatten()
    predictions7 += prediction7 / NFOLDS
    
    del model1, model2, prediction6, prediction7
    gc.collect()
    
    print(f'======== 224 Binary FOLD:{fold} inference ========')
    model = Model(model_name='swin_large_patch4_window7_224_in22k', num_classes=1)
    model.model.head = nn.Sequential(
        nn.Linear(model.model.head.in_features, 128),
        nn.Dropout(0.0),
        nn.Linear(128, 64),
        nn.Linear(64, 1)
    )
    model.load_state_dict(torch.load(f'/kaggle/input/petfinder2-binary-classifier-weight/fold_{fold}.pth'))
    model.to(device)
    prediction = inference_fn2(test_dl, model, device).flatten()
    binary += prediction / NFOLDS
    
    del model, prediction
    gc.collect()

coorabi['pred6'] = predictions6
coorabi['pred7'] = predictions7
coorabi['binary1'] = binary

!rm -r /kaggle/working/crop_images
del test_ds, test_dl, predictions6, predictions7, binary
gc.collect()

In [ ]:
#384x384
#5folds
#center crop
#binary classifier
test_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')
test_ids = test_df.Id.to_list()
os.chdir('/kaggle/working')
save_dir = f'/kaggle/working/crop_images/'
os.makedirs(save_dir, exist_ok=True)

for n, image_id in tqdm(enumerate(test_ids)):
    orig_image = cv2.imread(f'/kaggle/input/petfinder-pawpularity-score/test/{image_id}.jpg')
    orig_image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
    height = orig_image.shape[0]
    width = orig_image.shape[1]
    xc = width // 2
    yc = height // 2
    r = np.minimum(xc, yc)
    xmin = np.maximum(xc-r, 0)
    ymin = np.maximum(yc-r, 0)
    xmax = np.minimum(xc+r, width)
    ymax = np.minimum(yc+r, height)
    crop_img = orig_image[ymin:ymax, xmin:xmax, :]
    crop_img = cv2.resize(crop_img, (sz2, sz2)).astype(np.uint8)
    np.save(save_dir + f'{image_id}', crop_img)

test_ds = Dataset2(df=coorabi, transform=None)
test_dl = DataLoader(dataset=test_ds, batch_size=64, shuffle=False, num_workers=2)
target_cols = np.arange(1, 101)
predictions8 = 0
predictions9 = 0
binary = 0

for fold in range(NFOLDS):
    print(f'======== 384 center FOLD:{fold} inference ========')
    
    #SwinT type1: 100 outputs
    model1 = Model(model_name='swin_large_patch4_window12_384_in22k', num_classes=100)
    model1.load_state_dict(torch.load(f'/kaggle/input/petfinder2-swint1-weight-077/swint1_fold_{fold}.pth'))
    model1.to(device)
    prediction8 = inference_fn1(test_dl, model1, device)
    predictions8 += (target_cols.reshape(-1, 100) * prediction8.reshape(-1, 100)).sum(axis=1) / NFOLDS
    
    #SwinT type2: 1 output
    model2 = Model(model_name='swin_large_patch4_window12_384_in22k', num_classes=1)
    model2.model.head = nn.Sequential(
        nn.Linear(model2.model.head.in_features, 128),
        nn.Dropout(0.0),
        nn.Linear(128, 64),
        nn.Linear(64, 1)
    )
    model2.load_state_dict(torch.load(f'/kaggle/input/petfinder2-swint2-weight-037/swint2_fold_{fold}.pth'))
    model2.to(device)
    prediction9 = inference_fn2(test_dl, model2, device).flatten()
    predictions9 += prediction9 / NFOLDS
    
    del model1, model2, prediction8, prediction9
    gc.collect()
    
    print(f'======== 384 Binary FOLD:{fold} inference ========')
    model = Model(model_name='swin_large_patch4_window12_384_in22k', num_classes=1)
    model.model.head = nn.Sequential(
        nn.Linear(model.model.head.in_features, 128),
        nn.Dropout(0.0),
        nn.Linear(128, 64),
        nn.Linear(64, 1)
    )
    model.load_state_dict(torch.load(f'/kaggle/input/petfinder2-binary-classifier-weight2/fold_{fold}.pth'))
    model.to(device)
    prediction = inference_fn2(test_dl, model, device).flatten()
    binary += prediction / NFOLDS
    
    del model, prediction
    gc.collect()
    
coorabi['pred8'] = predictions8
coorabi['pred9'] = predictions9
coorabi['binary2'] = binary

!rm -r /kaggle/working/crop_images
del test_ds, test_dl, predictions8, predictions9, binary
gc.collect()

In [ ]:
coorabi['Pawpularity'] = ((coorabi['pred1'] * 0.275 + coorabi['pred2'] * 0.225 + coorabi['pred3'] * 0.275 + coorabi['pred4'] * 0.225) * 0.75 + coorabi['pred5'] * 0.25) * 0.6 + (coorabi['pred6'] * 0.22 + coorabi['pred7'] * 0.18 + coorabi['pred8'] * 0.33 + coorabi['pred9'] * 0.27) * 0.4
coorabi['binary'] = (coorabi['binary1'] + coorabi['binary2']) / 2

In [ ]:
coorabi

# Ensemble

In [ ]:
final = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
final

In [ ]:
final['Pawpularity'] = karunru['Pawpularity'].values * 0.5 + coorabi['Pawpularity'].values * 0.5
final['binary'] = coorabi['binary'].values
final

# PostProcess

In [ ]:
idx = final[(final['binary'] > 51) & (final['Pawpularity'] < 49)].index

In [ ]:
final.loc[idx, 'Pawpularity'] = final.loc[idx, 'Pawpularity'].values * 0.1 + final.loc[idx, 'binary'].values * 0.9

In [ ]:
final

In [ ]:
final[['Id', 'Pawpularity']].to_csv('/kaggle/working/submission.csv', index=False)